# tenk10k pipeline

In [1]:
import glob
import pandas as pd
import scanpy as sc
import scanpy.external as sce

In [2]:
# Output directory
out_dir = "/directflow/SCCGGroupShare/projects/blabow/tenk10k_phase1/data_processing/scanpy/output/integrated_objects/"

# Filtered object directory
scanpy_dir = "/directflow/SCCGGroupShare/projects/blabow/tenk10k_phase1/data_processing/scanpy/output/scanpy_objects_w_metadata/"

In [3]:
# extract all files
scanpy_files = glob.glob(scanpy_dir+"S*")

# combine anndata objects by samples
datasets=[]
for file in scanpy_files:
    adata = sc.read(file)
    datasets.append(adata)
adata = datasets[0].concatenate(*datasets[1:])
# adata = datasets[0].concatenate(datasets[1:]) # This works 

In [ ]:
# Notes: Making the concatenation work with higher number of samples
# more memory efficient (doesn't make 2 copies of each annData object)??
# adata = sc.read(scanpy_files[0])
# for file in scanpy_files[1:]:
#     adata.concatenate(sc.read(file)
# maybe I can use AnnCollection rather than concatenating all the datasets?
# this will create a concatenated view of the data without copying anything and should be more memory efficient
# from anndata.experimental.multi_files import AnnCollection
# Anncollection(datasets)

In [4]:
# Load gencode (v44) gene info file
gencode_text_file = '/directflow/SCCGGroupShare/projects/anncuo/reference_data/gencode.v44.basic.annotation_df.txt'
gene_info = pd.read_csv(gencode_text_file)

# Add gene info to AnnData
# set gene ids as indices
adata.var.index = [gene for gene in adata.var['gene_ids']]
genes_cellranger = adata.var.index
# Remove gene version
gene_info.index = [gene.split(".")[0] for gene in gene_info['gene_id']]
# Only consider genes in AnnData object
gene_info_df = gene_info[gene_info.index.isin(genes_cellranger)]
# Add info
adata.var = pd.concat([adata.var,gene_info_df], axis=1)

In [5]:
# write
out_file = f'{out_dir}/240_libraries_concatenated_gene_info.h5ad'
adata.write(out_file)

/share/ScratchGeneral/anncuo/jupyter/conda_notebooks/envs/cellregmap_notebook/lib/python3.7/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'celltypist_predicted_labels' as categorical
/share/ScratchGeneral/anncuo/jupyter/conda_notebooks/envs/cellregmap_notebook/lib/python3.7/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'celltypist_over_clustering' as categorical
/share/ScratchGeneral/anncuo/jupyter/conda_notebooks/envs/cellregmap_notebook/lib/pyt

/share/ScratchGeneral/anncuo/jupyter/conda_notebooks/envs/cellregmap_notebook/lib/python3.7/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'source' as categorical
/share/ScratchGeneral/anncuo/jupyter/conda_notebooks/envs/cellregmap_notebook/lib/python3.7/site-packages/anndata/_core/anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'type' as categorical
/share/ScratchGeneral/anncuo/jupyter/conda_notebooks/envs/cellregmap_notebook/lib/python3.7/site-packages/anndata/_core/anndata.

# Save list of all the cell types in the object, used to split the object later in the pipeline

In [ ]:
for ct in adata.obs['wg2_scpred_prediction'].unique():
    print(ct)

In [ ]:
# save list of unique cell types to a .txt file 
unique_cell_types = adata.obs['wg2_scpred_prediction'].unique()

# Write unique cell types to a text file
with open(f'{out_dir}unique_cell_types_wg2_scpred.txt', 'w') as f:
    for cell_type in unique_cell_types:
        f.write(str(cell_type) + '\n')

# QC: Checking that the new object has all data compared to the previous run

### 240 samples object

In [6]:
adata_240 = adata

In [14]:
adata_240.obs[adata_240.obs['tob_id'] == 'TOB1067'][['tob_id', 'cpg_id', 'cpg_id_old']]

,tob_id,cpg_id,cpg_id_old
AAACCCAAGGTAAGTT_S0033a-4,TOB1067,CPG319350,CPG10835
AAACGAACATGCGTGC_S0033a-4,TOB1067,CPG319350,CPG10835
AAACGAATCTTCGCTG_S0033a-4,TOB1067,CPG319350,CPG10835
AAAGAACCATGAGTAA_S0033a-4,TOB1067,CPG319350,CPG10835
AAAGAACTCAACTCTT_S0033a-4,TOB1067,CPG319350,CPG10835
...,...,...,...
TTTGATCCAAGGATGC_S0033c-207,TOB1067,CPG319350,CPG10835
TTTGATCCACAACGTT_S0033c-207,TOB1067,CPG319350,CPG10835
TTTGGAGGTGTTTACG_S0033c-207,TOB1067,CPG319350,CPG10835
TTTGGTTAGGTGATAT_S0033c-207,TOB1067,CPG319350,CPG10835


In [ ]:
adata_240.obs[adata_240.obs['tob_id'] == 'TOB1067'][['tob_id', 'cpg_id', 'cpg_id_old']]

In [3]:
# adata_240 = sc.read(f'{out_dir}/240_libraries_concatenated_gene_info.h5ad')

In [7]:
adata_240.obs.shape

(5084027, 40)

In [8]:
# how many pools?
adata_240.obs['batch'].nunique()


240

In [9]:
# how many individuals?
adata_240.obs['sequencing_library'].nunique()

240

In [10]:
adata_240.obs['individual'].nunique()

1812

In [8]:
adata_240.obs['individual'].unique()

['CPG248914', 'CPG248146', 'CPG247312', 'CPG248690', 'CPG247387', ..., 'CPG255943', 'CPG249631', 'CPG256388', 'CPG255927', 'CPG251512']
Length: 1728
Categories (1728, object): ['CPG247296', 'CPG247304', 'CPG247312', 'CPG247320', ..., 'CPG317990', 'donor6_S0049', 'donor7_S0049', 'donor7_S0053']

In [9]:
adata_240.obs.columns

Index(['cellbender_background_fraction', 'cellbender_cell_probability',
       'cellbender_cell_size', 'cellbender_droplet_efficiency',
       'celltypist_predicted_labels', 'celltypist_over_clustering',
       'celltypist_majority_voting', 'celltypist_conf_score', 'wg2_sample',
       'wg2_nCount_RNA', 'wg2_nFeature_RNA', 'wg2_percent_mt',
       'wg2_azimuth_predicted_celltype_l2',
       'wg2_azimuth_predicted_celltype_l2_score', 'wg2_scpred_prediction',
       'Vireo_Individual_Assignment', 'Vireo_DropletType',
       'scDblFinder_DropletType', 'scDblFinder_Score', 'scds_score',
       'scds_DropletType', 'MajoritySinglet_DropletType',
       'MajoritySinglet_Individual_Assignment', 'n_genes', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'original_barcode',
       'new_cell_name', 'sequencing_library', 'individual', 'cohort', 'ct_id',
       'cpg_id', 'onek1k_id', 'cpg_id_old', 'tob_id', 'onek1k_donor', 'batch'],
      dtype='object')

In [10]:
with pd.option_context('display.max_rows', None,): 
    print(adata_240.obs[['sequencing_library', 'individual']].value_counts())

sequencing_library  individual  
S0040_repeat        CPG247312       7420
S0055               CPG247700       6739
S0062               CPG305953       6028
S0052               CPG247924       5999
S0072               CPG256271       5844
S0067               CPG256347       5742
S0058_re            CPG248021       5741
S0061               CPG256446       5629
S0151               CPG256453       5541
S0105               CPG305946       5539
S0152               CPG256438       5478
S0139               CPG255638       5408
S0066               CPG255687       5365
S0070               CPG256222       5328
S0132               CPG255539       5317
S0063               CPG256008       5315
S0073               CPG255257       5280
S0042_repeat        CPG247494       5097
S0153               CPG255489       5096
S0064               CPG255497       5070
S0052               CPG247635       5068
S0134               CPG256057       5032
S0069               CPG256032       4951
S0058_re            CPG2

In [12]:
[print(i) for i in adata_240.obs['individual'].unique()]

CPG248914
CPG248146
CPG247312
CPG248690
CPG247387
CPG247841
CPG248880
CPG248286
CPG248054
CPG248377
CPG247981
CPG247544
CPG248716
CPG313536
CPG313551
CPG311720
CPG313544
CPG317768
CPG317735
CPG311746
CPG313890
CPG313957
CPG311779
CPG317917
CPG311795
CPG311753
CPG313569
CPG311761
CPG313874
CPG309930
CPG313908
CPG311787
CPG313916
CPG317750
CPG311738
CPG317776
CPG317743
CPG313510
CPG313528
CPG309112
CPG313866
CPG309104
CPG310896
CPG313841
CPG313817
CPG313767
CPG310862
CPG310847
CPG313486
CPG313759
CPG313627
CPG310888
CPG313858
CPG310797
CPG313494
CPG313643
CPG313825
CPG313742
CPG310870
CPG313833
CPG310854
CPG255448
CPG254870
CPG251579
CPG255786
CPG250381
CPG253815
CPG253658
CPG250472
CPG314138
CPG314765
CPG314732
CPG314427
CPG314393
CPG314401
CPG317966
CPG314807
CPG314179
CPG314385
CPG314575
CPG314781
CPG314112
CPG317958
CPG314120
CPG314815
CPG314823
CPG314419
CPG314740
CPG313668
CPG313882
CPG314799
CPG314773
CPG315010
CPG315028
CPG312025
CPG309427
CPG314997
CPG314872
CPG311977
CPG314831


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [28]:
outs_240214 = [
    "S0056a",
    "S0056b",
    "S0057a",
    "S0057b",
    "S0058_re",
    "S0059a",
    "S0059b",
    "S0102",
    "S0104",
    "S0146",
    "S0148",
    "S0149",
    "S0150",
    "S0151",
    "S0152",
    "S0153"
]
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(adata_240.obs[adata_240.obs['sequencing_library'].isin(outs_240214)][['sequencing_library', 'individual']].drop_duplicates())

                             sequencing_library individual
AAACCCAAGACAGTCG_S0149-44                 S0149  CPG251678
AAACCCAAGACCGTTT_S0149-44                 S0149  CPG253112
AAACCCAAGAGCCCAA_S0149-44                 S0149  CPG253013
AAACCCAAGCCTAGGA_S0149-44                 S0149  CPG255802
AAACCCACAAATACGA_S0149-44                 S0149  CPG306050
AAACCCACAATGAAAC_S0149-44                 S0149  CPG249771
AAACCCAGTACAGTCT_S0149-44                 S0149  CPG254375
AAACCCAAGAGGTGCT_S0057a-45               S0057a  CPG248740
AAACCCAAGCACCGTC_S0057a-45               S0057a  CPG247999
AAACCCAAGGGCTGAT_S0057a-45               S0057a  CPG248484
AAACCCAAGGTTCCAT_S0057a-45               S0057a  CPG248575
AAACCCAAGTAGAATC_S0057a-45               S0057a  CPG248468
AAACCCAAGTATGAGT_S0057a-45               S0057a  CPG248864
AAACCCACAGGCCTGT_S0057a-45               S0057a  CPG249136
AAACCCAGTACGTGAG_S0057a-45               S0057a  CPG248310
AAACCCAGTCAGATTC_S0057a-45               S0057a  CPG2482

In [30]:
len(adata_240.obs['individual'].unique())

1728

In [31]:
len(adata_240.obs['cpg_id'].unique())

1728

In [35]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print (adata_240.obs[['sequencing_library', 'individual']].drop_duplicates().groupby('sequencing_library').size())

sequencing_library
S0001a           24
S0001b           24
S0001c           24
S0002a           21
S0002b           21
S0002c           21
S0003a           23
S0003b           23
S0003c           23
S0004a           26
S0004b           26
S0004c           26
S0005a           20
S0005b           20
S0005c           20
S0006a           25
S0006b           25
S0006c           25
S0007a           22
S0007b           22
S0007c           23
S0008a           20
S0008b           20
S0008c           20
S0009a           26
S0009b           26
S0009c           26
S0010a           27
S0010b           27
S0010c           27
S0011a           24
S0011b           24
S0011c           24
S0012a           26
S0012b           26
S0012c           26
S0013a           21
S0013b           21
S0013c           21
S0014a           25
S0014b           25
S0014c           25
S0015a           26
S0015b           26
S0015c           26
S0016a           23
S0016b           23
S0016c           23
S0017b           22
S

In [36]:
len(adata_240.obs['cpg_id_old'].unique())

893

In [37]:
cpg_ids_prev = {"CPG1743",
"CPG1743",
"CPG5264",
"CPG5264",
"CPG4655",
"CPG4655",
"CPG1792",
"CPG1792",
"CPG1818",
"CPG1818",
"CPG1768",
"CPG1768",
"CPG4408",
"CPG4408",
"CPG6247",
"CPG6247",
"CPG1719",
"CPG1719",
"CPG4424",
"CPG4424",
"CPG1750",
"CPG4432",
"CPG4416",
"CPG4531",
"CPG4630",
"CPG6254",
"CPG4556",
"CPG11205",
"CPG1727",
"CPG1826",
"CPG4663",
"CPG4002",
"CPG4671",
"CPG5272",
"CPG4564",
"CPG1735",
"CPG1743",
"CPG5264",
"CPG4655",
"CPG1768",
"CPG1792",
"CPG6247",
"CPG4408",
"CPG1818",
"CPG1719",
"CPG4432",
"CPG1750",
"CPG4424",
"CPG4531",
"CPG4416",
"CPG4556",
"CPG4630",
"CPG1826",
"CPG4663",
"CPG1727",
"CPG11205",
"CPG4002",
"CPG4671",
"CPG1735",
"CPG5272",
"CPG4564",
"CPG6254",
"CPG5264",
"CPG1743",
"CPG4655",
"CPG1792",
"CPG4408",
"CPG1818",
"CPG1768",
"CPG6247",
"CPG1750",
"CPG1719",
"CPG4432",
"CPG4424",
"CPG4416",
"CPG4531",
"CPG6254",
"CPG4630",
"CPG4556",
"CPG4663",
"CPG1826",
"CPG11205",
"CPG1727",
"CPG4671",
"CPG5272",
"CPG4002",
"CPG1735",
"CPG4564",
"CPG4572",
"CPG3822",
"CPG5470",
"CPG5504",
"CPG5512",
"CPG4622",
"CPG11197",
"CPG3814",
"CPG3954",
"CPG4648",
"CPG3889",
"CPG7518",
"CPG11155",
"CPG7138",
"CPG7161",
"CPG7534",
"CPG7146",
"CPG7542",
"CPG7567",
"CPG7153",
"CPG7484",
"CPG7476",
"CPG7492",
"CPG3822",
"CPG4572",
"CPG5470",
"CPG5504",
"CPG5512",
"CPG11197",
"CPG4622",
"CPG3814",
"CPG3954",
"CPG4648",
"CPG3889",
"CPG7518",
"CPG11155",
"CPG7534",
"CPG7138",
"CPG7542",
"CPG7161",
"CPG7153",
"CPG7146",
"CPG7567",
"CPG7484",
"CPG7476",
"CPG7492",
"CPG4572",
"CPG3822",
"CPG5470",
"CPG5504",
"CPG5512",
"CPG11197",
"CPG3814",
"CPG3954",
"CPG4622",
"CPG4648",
"CPG3889",
"CPG11155",
"CPG7138",
"CPG7534",
"CPG7518",
"CPG7153",
"CPG7542",
"CPG7161",
"CPG7567",
"CPG7476",
"CPG7484",
"CPG7492",
"CPG7146",
"CPG3673",
"CPG3707",
"CPG3715",
"CPG3459",
"CPG3632",
"CPG3699",
"CPG8615",
"CPG10603",
"CPG3665",
"CPG9456",
"CPG10595",
"CPG10579",
"CPG3608",
"CPG3640",
"CPG8581",
"CPG10611",
"CPG3590",
"CPG3657",
"CPG9449",
"CPG8607",
"CPG10587",
"CPG3681",
"CPG3624",
"CPG10637",
"CPG8599",
"CPG8573",
"CPG3616",
"CPG3707",
"CPG3715",
"CPG3673",
"CPG3632",
"CPG3459",
"CPG3699",
"CPG8615",
"CPG3665",
"CPG10603",
"CPG9456",
"CPG10579",
"CPG3608",
"CPG3640",
"CPG10595",
"CPG8581",
"CPG9449",
"CPG3657",
"CPG3590",
"CPG10611",
"CPG10637",
"CPG3681",
"CPG10587",
"CPG8607",
"CPG3624",
"CPG8599",
"CPG3616",
"CPG8573",
"CPG3707",
"CPG3673",
"CPG3715",
"CPG3632",
"CPG3459",
"CPG3699",
"CPG10579",
"CPG10603",
"CPG8615",
"CPG3665",
"CPG9456",
"CPG3608",
"CPG3640",
"CPG10611",
"CPG10595",
"CPG3657",
"CPG8581",
"CPG3590",
"CPG8607",
"CPG3681",
"CPG3624",
"CPG10587",
"CPG9449",
"CPG10637",
"CPG8599",
"CPG8573",
"CPG3616",
"CPG3541",
"CPG3939",
"CPG3558",
"CPG5397",
"CPG3509",
"CPG3491",
"CPG3517",
"CPG3566",
"CPG5413",
"CPG4614",
"CPG3475",
"CPG4747",
"CPG3921",
"CPG4739",
"CPG3483",
"CPG5439",
"CPG3897",
"CPG3574",
"CPG3913",
"CPG4333",
"CPG5421",
"CPG3905",
"CPG3582",
"CPG3525",
"CPG3467",
"CPG5405",
"CPG3541",
"CPG3939",
"CPG3517",
"CPG3558",
"CPG3491",
"CPG5397",
"CPG3566",
"CPG3509",
"CPG3475",
"CPG5413",
"CPG4739",
"CPG3921",
"CPG4614",
"CPG4747",
"CPG5439",
"CPG3913",
"CPG3897",
"CPG3574",
"CPG3483",
"CPG4333",
"CPG3905",
"CPG5421",
"CPG3467",
"CPG3525",
"CPG5405",
"CPG3582",
"CPG3541",
"CPG5397",
"CPG3939",
"CPG3558",
"CPG5413",
"CPG3517",
"CPG3509",
"CPG3491",
"CPG3566",
"CPG3475",
"CPG3921",
"CPG4614",
"CPG3574",
"CPG4747",
"CPG5439",
"CPG4739",
"CPG3897",
"CPG5421",
"CPG3913",
"CPG3483",
"CPG4333",
"CPG3582",
"CPG3905",
"CPG3525",
"CPG3467",
"CPG5405",
"CPG8243",
"CPG9993",
"CPG8169",
"CPG8250",
"CPG10017",
"CPG8185",
"CPG9985",
"CPG8235",
"CPG9936",
"CPG8177",
"CPG8268",
"CPG8193",
"CPG9969",
"CPG9951",
"CPG9977",
"CPG8284",
"CPG9902",
"CPG9910",
"CPG10025",
"CPG9944",
"CPG9928",
"CPG8243",
"CPG9993",
"CPG8169",
"CPG8250",
"CPG10017",
"CPG8185",
"CPG9936",
"CPG9985",
"CPG8235",
"CPG8193",
"CPG8177",
"CPG8268",
"CPG9969",
"CPG9977",
"CPG9951",
"CPG8284",
"CPG9902",
"CPG9910",
"CPG9944",
"CPG10025",
"CPG9928",
"CPG8243",
"CPG9993",
"CPG8169",
"CPG8250",
"CPG8185",
"CPG10017",
"CPG9985",
"CPG8235",
"CPG9936",
"CPG8193",
"CPG8268",
"CPG9969",
"CPG8284",
"CPG8177",
"CPG9977",
"CPG9951",
"CPG9902",
"CPG9910",
"CPG10025",
"CPG9944",
"CPG9928",
"CPG8433",
"CPG4770",
"CPG4812",
"CPG4119",
"CPG4127",
"CPG4788",
"CPG4796",
"CPG4754",
"CPG8383",
"CPG4093",
"CPG4820",
"CPG4101",
"CPG8359",
"CPG4804",
"CPG8318",
"CPG8425",
"CPG4762",
"CPG8334",
"CPG4945",
"CPG8342",
"CPG8367",
"CPG8409",
"CPG8391",
"CPG8326",
"CPG8417",
"CPG8433",
"CPG4770",
"CPG4788",
"CPG4812",
"CPG4127",
"CPG4119",
"CPG4796",
"CPG4754",
"CPG4093",
"CPG4820",
"CPG8383",
"CPG4101",
"CPG4804",
"CPG8359",
"CPG4762",
"CPG8318",
"CPG4945",
"CPG8425",
"CPG8334",
"CPG8342",
"CPG8367",
"CPG8409",
"CPG8391",
"CPG8326",
"CPG8417",
"CPG8433",
"CPG4770",
"CPG4788",
"CPG4812",
"CPG4127",
"CPG4119",
"CPG4754",
"CPG4101",
"CPG4093",
"CPG4796",
"CPG8359",
"CPG8383",
"CPG4820",
"CPG4762",
"CPG4804",
"CPG8425",
"CPG8318",
"CPG4945",
"CPG8409",
"CPG8334",
"CPG8342",
"CPG8367",
"CPG8391",
"CPG8326",
"CPG8417",
"CPG2667",
"CPG2675",
"CPG2741",
"CPG2733",
"CPG9662",
"CPG2725",
"CPG2717",
"CPG2691",
"CPG2683",
"CPG9209",
"CPG9696",
"CPG9233",
"CPG9241",
"CPG2972",
"CPG9688",
"CPG2964",
"CPG10918",
"CPG9191",
"CPG9670",
"CPG2956",
"CPG3046",
"CPG10900",
"CPG7468",
"CPG2667",
"CPG2675",
"CPG2741",
"CPG2733",
"CPG9662",
"CPG2717",
"CPG2725",
"CPG2683",
"CPG9209",
"CPG9233",
"CPG2691",
"CPG9696",
"CPG10918",
"CPG9241",
"CPG2972",
"CPG9191",
"CPG2964",
"CPG9688",
"CPG9670",
"CPG3046",
"CPG2956",
"CPG10900",
"CPG7468",
"CPG2667",
"CPG2675",
"CPG2741",
"CPG2733",
"CPG9662",
"CPG2691",
"CPG2683",
"CPG2725",
"CPG2717",
"CPG9209",
"CPG9233",
"CPG9688",
"CPG2972",
"CPG9696",
"CPG10918",
"CPG2964",
"CPG9241",
"CPG9191",
"CPG3046",
"CPG9670",
"CPG10900",
"CPG2956",
"CPG7468",
"CPG9225",
"CPG10306",
"CPG8656",
"CPG2337",
"CPG7864",
"CPG8714",
"CPG7872",
"CPG8680",
"CPG10314",
"CPG7849",
"CPG8904",
"CPG7831",
"CPG2360",
"CPG10298",
"CPG2311",
"CPG2345",
"CPG2287",
"CPG2352",
"CPG2162",
"CPG2188",
"CPG7799",
"CPG2295",
"CPG2329",
"CPG2261",
"CPG2279",
"CPG2303",
"CPG10306",
"CPG8656",
"CPG7864",
"CPG7872",
"CPG2337",
"CPG8714",
"CPG10314",
"CPG7849",
"CPG7831",
"CPG10298",
"CPG8680",
"CPG8904",
"CPG2287",
"CPG2345",
"CPG2311",
"CPG2352",
"CPG2360",
"CPG2329",
"CPG2188",
"CPG2162",
"CPG2295",
"CPG2261",
"CPG7799",
"CPG2279",
"CPG2303",
"CPG10306",
"CPG7864",
"CPG8656",
"CPG7872",
"CPG2337",
"CPG8714",
"CPG8680",
"CPG7849",
"CPG10314",
"CPG7831",
"CPG8904",
"CPG10298",
"CPG2311",
"CPG2329",
"CPG2345",
"CPG2352",
"CPG2287",
"CPG2188",
"CPG2162",
"CPG7799",
"CPG2295",
"CPG2303",
"CPG2360",
"CPG2279",
"CPG2261",
"CPG1321",
"CPG2428",
"CPG1313",
"CPG3376",
"CPG3319",
"CPG3368",
"CPG3350",
"CPG3335",
"CPG3327",
"CPG3434",
"CPG3343",
"CPG1339",
"CPG8060",
"CPG1347",
"CPG8128",
"CPG1107",
"CPG8144",
"CPG8078",
"CPG8102",
"CPG8136",
"CPG11114",
"CPG3442",
"CPG8052",
"CPG8110",
"CPG8219",
"CPG8086",
"CPG8094",
"CPG8227",
"CPG1321",
"CPG2428",
"CPG3376",
"CPG1313",
"CPG3319",
"CPG3350",
"CPG3368",
"CPG3335",
"CPG3434",
"CPG3327",
"CPG3343",
"CPG1347",
"CPG1339",
"CPG8060",
"CPG1107",
"CPG8128",
"CPG8078",
"CPG8144",
"CPG8136",
"CPG8102",
"CPG11114",
"CPG8110",
"CPG8052",
"CPG3442",
"CPG8219",
"CPG8086",
"CPG8227",
"CPG8094",
"CPG1321",
"CPG2428",
"CPG1313",
"CPG3376",
"CPG3319",
"CPG3350",
"CPG3368",
"CPG3434",
"CPG3335",
"CPG3327",
"CPG3343",
"CPG1339",
"CPG8060",
"CPG1347",
"CPG1107",
"CPG8128",
"CPG8078",
"CPG8144",
"CPG8136",
"CPG8102",
"CPG11114",
"CPG3442",
"CPG8110",
"CPG8052",
"CPG8219",
"CPG8094",
"CPG8086",
"CPG8227",
"CPG2642",
"CPG2584",
"CPG2626",
"CPG2998",
"CPG2576",
"CPG5207",
"CPG2550",
"CPG2592",
"CPG2535",
"CPG2634",
"CPG2568",
"CPG6239",
"CPG5249",
"CPG5223",
"CPG5199",
"CPG2618",
"CPG2600",
"CPG6205",
"CPG6536",
"CPG5256",
"CPG5215",
"CPG6221",
"CPG2980",
"CPG6197",
"CPG6528",
"CPG5785",
"CPG2543",
"CPG2584",
"CPG2998",
"CPG2626",
"CPG2642",
"CPG2592",
"CPG2550",
"CPG5207",
"CPG2576",
"CPG2634",
"CPG2535",
"CPG5223",
"CPG2568",
"CPG6239",
"CPG5249",
"CPG5199",
"CPG2618",
"CPG2600",
"CPG5215",
"CPG6536",
"CPG6205",
"CPG5256",
"CPG6221",
"CPG2980",
"CPG6197",
"CPG6528",
"CPG5785",
"CPG2543",
"CPG2584",
"CPG2642",
"CPG2626",
"CPG2998",
"CPG2592",
"CPG2576",
"CPG5207",
"CPG2550",
"CPG2535",
"CPG5223",
"CPG2634",
"CPG2568",
"CPG6239",
"CPG5249",
"CPG6536",
"CPG2600",
"CPG5256",
"CPG5215",
"CPG5199",
"CPG6205",
"CPG2618",
"CPG6221",
"CPG2980",
"CPG6197",
"CPG6528",
"CPG5785",
"CPG2543",
"CPG3855",
"CPG11007",
"CPG11064",
"CPG6056",
"CPG3806",
"CPG3848",
"CPG9225",
"CPG11023",
"CPG3863",
"CPG11015",
"CPG11056",
"CPG8045",
"CPG4010",
"CPG10983",
"CPG6064",
"CPG11049",
"CPG5389",
"CPG6122",
"CPG3830",
"CPG4465",
"CPG11031",
"CPG6114",
"CPG10991",
"CPG6072",
"CPG4440",
"CPG9217",
"CPG5496",
"CPG9183",
"CPG3855",
"CPG11007",
"CPG6056",
"CPG3848",
"CPG11064",
"CPG3806",
"CPG8045",
"CPG11023",
"CPG3863",
"CPG6064",
"CPG9225",
"CPG11015",
"CPG11056",
"CPG10983",
"CPG11049",
"CPG4010",
"CPG5389",
"CPG6122",
"CPG11031",
"CPG3830",
"CPG4465",
"CPG6072",
"CPG6114",
"CPG10991",
"CPG4440",
"CPG9217",
"CPG5496",
"CPG9183",
"CPG3855",
"CPG11007",
"CPG3848",
"CPG6056",
"CPG11064",
"CPG3806",
"CPG11023",
"CPG9225",
"CPG8045",
"CPG3863",
"CPG11015",
"CPG11049",
"CPG6064",
"CPG11056",
"CPG4010",
"CPG10983",
"CPG11031",
"CPG5389",
"CPG3830",
"CPG6122",
"CPG4465",
"CPG6072",
"CPG4440",
"CPG6114",
"CPG10991",
"CPG9217",
"CPG5496",
"CPG9183",
"CPG5298",
"CPG4374",
"CPG5306",
"CPG5280",
"CPG5546",
"CPG6130",
"CPG6189",
"CPG2014",
"CPG5736",
"CPG5751",
"CPG927",
"CPG2030",
"CPG2006",
"CPG2048",
"CPG6106",
"CPG950",
"CPG6510",
"CPG5538",
"CPG5447",
"CPG5728",
"CPG1982",
"CPG2055",
"CPG1123",
"CPG2022",
"CPG6171",
"CPG943",
"CPG5298",
"CPG4374",
"CPG5306",
"CPG5546",
"CPG5280",
"CPG6130",
"CPG6189",
"CPG5736",
"CPG2014",
"CPG2048",
"CPG943",
"CPG5751",
"CPG927",
"CPG2030",
"CPG2006",
"CPG6106",
"CPG950",
"CPG5447",
"CPG5538",
"CPG5728",
"CPG6510",
"CPG1982",
"CPG2022",
"CPG2055",
"CPG6171",
"CPG1123",
"CPG5298",
"CPG4374",
"CPG5306",
"CPG5280",
"CPG6130",
"CPG5546",
"CPG6189",
"CPG5736",
"CPG2014",
"CPG2048",
"CPG5751",
"CPG6106",
"CPG2030",
"CPG927",
"CPG2006",
"CPG943",
"CPG5447",
"CPG950",
"CPG6510",
"CPG5538",
"CPG1982",
"CPG5728",
"CPG2055",
"CPG2022",
"CPG1123",
"CPG6171",
"CPG6908",
"CPG9092",
"CPG6312",
"CPG10520",
"CPG6759",
"CPG6304",
"CPG5611",
"CPG5520",
"CPG9076",
"CPG9084",
"CPG10496",
"CPG9050",
"CPG6890",
"CPG10504",
"CPG10538",
"CPG6296",
"CPG10363",
"CPG9068",
"CPG10512",
"CPG5231",
"CPG9118",
"CPG9100",
"CPG5579",
"CPG6767",
"CPG10371",
"CPG6882",
"CPG6908",
"CPG9092",
"CPG6312",
"CPG10520",
"CPG6304",
"CPG6759",
"CPG9076",
"CPG10504",
"CPG9084",
"CPG10496",
"CPG5520",
"CPG6296",
"CPG9050",
"CPG10512",
"CPG5611",
"CPG10363",
"CPG9068",
"CPG5231",
"CPG10538",
"CPG6890",
"CPG6767",
"CPG9100",
"CPG9118",
"CPG5579",
"CPG6882",
"CPG10371",
"CPG6908",
"CPG9092",
"CPG6312",
"CPG10520",
"CPG6304",
"CPG6759",
"CPG10504",
"CPG5520",
"CPG9084",
"CPG9076",
"CPG10496",
"CPG5611",
"CPG9050",
"CPG6296",
"CPG10538",
"CPG10363",
"CPG10512",
"CPG6890",
"CPG9068",
"CPG5231",
"CPG9100",
"CPG9118",
"CPG6767",
"CPG5579",
"CPG10371",
"CPG6882",
"CPG5082",
"CPG5702",
"CPG5090",
"CPG5116",
"CPG5108",
"CPG5066",
"CPG5124",
"CPG4986",
"CPG5074",
"CPG5462",
"CPG5561",
"CPG6148",
"CPG6155",
"CPG4606",
"CPG4457",
"CPG5686",
"CPG5694",
"CPG6163",
"CPG8458",
"CPG5454",
"CPG1636",
"CPG8037",
"CPG4598",
"CPG4994",
"CPG4580",
"CPG4960",
"CPG5553",
"CPG5082",
"CPG5702",
"CPG5116",
"CPG5108",
"CPG5090",
"CPG5066",
"CPG8458",
"CPG5074",
"CPG6155",
"CPG5462",
"CPG5561",
"CPG5694",
"CPG5686",
"CPG6148",
"CPG4986",
"CPG5124",
"CPG5454",
"CPG4457",
"CPG1636",
"CPG4598",
"CPG6163",
"CPG4606",
"CPG8037",
"CPG4960",
"CPG5553",
"CPG4994",
"CPG4580",
"CPG5082",
"CPG5702",
"CPG5090",
"CPG5116",
"CPG5066",
"CPG5108",
"CPG6148",
"CPG5074",
"CPG5462",
"CPG5694",
"CPG6155",
"CPG5561",
"CPG8458",
"CPG5686",
"CPG4986",
"CPG5124",
"CPG6163",
"CPG4606",
"CPG4457",
"CPG5454",
"CPG4598",
"CPG1636",
"CPG4960",
"CPG8037",
"CPG4580",
"CPG4994",
"CPG5553",
"CPG1198",
"CPG1560",
"CPG1149",
"CPG1230",
"CPG1156",
"CPG1131",
"CPG1172",
"CPG1578",
"CPG1404",
"CPG8706",
"CPG1586",
"CPG10322",
"CPG7963",
"CPG7971",
"CPG1545",
"CPG7997",
"CPG1248",
"CPG8011",
"CPG7823",
"CPG10330",
"CPG1552",
"CPG8003",
"CPG7815",
"CPG1164",
"CPG7989",
"CPG7856",
"CPG7948",
"CPG8912",
"CPG1198",
"CPG1560",
"CPG1149",
"CPG1172",
"CPG1156",
"CPG1230",
"CPG1131",
"CPG8706",
"CPG1578",
"CPG1404",
"CPG1586",
"CPG10322",
"CPG7963",
"CPG1545",
"CPG7997",
"CPG7971",
"CPG1248",
"CPG10330",
"CPG8011",
"CPG7823",
"CPG1552",
"CPG8003",
"CPG7815",
"CPG1164",
"CPG7856",
"CPG7989",
"CPG7948",
"CPG8912",
"CPG1198",
"CPG1131",
"CPG1560",
"CPG1230",
"CPG1156",
"CPG1172",
"CPG1149",
"CPG1578",
"CPG8706",
"CPG1404",
"CPG7963",
"CPG1586",
"CPG10322",
"CPG7971",
"CPG7997",
"CPG1545",
"CPG8011",
"CPG1248",
"CPG7823",
"CPG10330",
"CPG8003",
"CPG1552",
"CPG7815",
"CPG1164",
"CPG7856",
"CPG7989",
"CPG7948",
"CPG8912",
"CPG7187",
"CPG7617",
"CPG7633",
"CPG7195",
"CPG7591",
"CPG7609",
"CPG7203",
"CPG9746",
"CPG7583",
"CPG7179",
"CPG7229",
"CPG9753",
"CPG7211",
"CPG7575",
"CPG11098",
"CPG11080",
"CPG11130",
"CPG3871",
"CPG11122",
"CPG11106",
"CPG9761",
"CPG11072",
"CPG9738",
"CPG11148",
"CPG9720",
"CPG7187",
"CPG7617",
"CPG7633",
"CPG7195",
"CPG7203",
"CPG7591",
"CPG7609",
"CPG11072",
"CPG7583",
"CPG7179",
"CPG7229",
"CPG7211",
"CPG9753",
"CPG7575",
"CPG11098",
"CPG3871",
"CPG11130",
"CPG11122",
"CPG11106",
"CPG11080",
"CPG9746",
"CPG9738",
"CPG9761",
"CPG11148",
"CPG9720",
"CPG7187",
"CPG7617",
"CPG7633",
"CPG7195",
"CPG7203",
"CPG7591",
"CPG9761",
"CPG7609",
"CPG7583",
"CPG7179",
"CPG7229",
"CPG7211",
"CPG9753",
"CPG7575",
"CPG11098",
"CPG11080",
"CPG11130",
"CPG11122",
"CPG3871",
"CPG11106",
"CPG9738",
"CPG11072",
"CPG9746",
"CPG11148",
"CPG9720",
"CPG836",
"CPG5603",
"CPG2758",
"CPG2782",
"CPG2709",
"CPG5892",
"CPG2816",
"CPG5991",
"CPG2808",
"CPG5595",
"CPG5777",
"CPG844",
"CPG5744",
"CPG5959",
"CPG5884",
"CPG2766",
"CPG5975",
"CPG5967",
"CPG5942",
"CPG5876",
"CPG5983",
"CPG2790",
"CPG836",
"CPG5603",
"CPG2758",
"CPG2709",
"CPG5892",
"CPG2782",
"CPG5983",
"CPG5991",
"CPG2808",
"CPG5595",
"CPG5777",
"CPG2816",
"CPG5744",
"CPG844",
"CPG5967",
"CPG5959",
"CPG2766",
"CPG5975",
"CPG5884",
"CPG5942",
"CPG5876",
"CPG2790",
"CPG4325",
"CPG778",
"CPG9399",
"CPG3111",
"CPG3020",
"CPG4309",
"CPG3087",
"CPG9381",
"CPG4291",
"CPG4317",
"CPG4234",
"CPG3053",
"CPG3012",
"CPG4242",
"CPG9415",
"CPG3095",
"CPG4259",
"CPG3038",
"CPG786",
"CPG3079",
"CPG9407",
"CPG4267",
"CPG794",
"CPG3004",
"CPG4275",
"CPG3061",
"CPG4325",
"CPG3111",
"CPG778",
"CPG9399",
"CPG3020",
"CPG3087",
"CPG9381",
"CPG9415",
"CPG4309",
"CPG3053",
"CPG4234",
"CPG4317",
"CPG4291",
"CPG4242",
"CPG3095",
"CPG3012",
"CPG4259",
"CPG3038",
"CPG786",
"CPG4267",
"CPG3079",
"CPG3004",
"CPG9407",
"CPG794",
"CPG4275",
"CPG3061",
"CPG4325",
"CPG9399",
"CPG778",
"CPG3111",
"CPG3020",
"CPG9381",
"CPG4242",
"CPG3087",
"CPG4309",
"CPG4291",
"CPG4317",
"CPG9415",
"CPG3053",
"CPG3012",
"CPG4234",
"CPG3095",
"CPG3038",
"CPG4267",
"CPG4259",
"CPG3079",
"CPG786",
"CPG9407",
"CPG3004",
"CPG794",
"CPG3061",
"CPG4275",
"CPG7070",
"CPG7104",
"CPG1180",
"CPG7716",
"CPG4077",
"CPG4044",
"CPG7062",
"CPG7245",
"CPG7112",
"CPG7294",
"CPG7096",
"CPG7328",
"CPG7054",
"CPG4382",
"CPG7286",
"CPG4069",
"CPG7278",
"CPG7682",
"CPG7252",
"CPG7260",
"CPG7336",
"CPG7302",
"CPG4036",
"CPG7674",
"CPG4390",
"CPG3996",
"CPG7310",
"CPG7237",
"CPG7070",
"CPG7104",
"CPG1180",
"CPG7062",
"CPG7716",
"CPG7245",
"CPG7294",
"CPG4044",
"CPG4077",
"CPG7112",
"CPG7286",
"CPG7328",
"CPG7096",
"CPG7054",
"CPG4382",
"CPG7278",
"CPG4069",
"CPG7252",
"CPG7682",
"CPG7336",
"CPG7260",
"CPG7302",
"CPG7674",
"CPG4390",
"CPG4036",
"CPG3996",
"CPG7310",
"CPG7237",
"CPG3236",
"CPG3285",
"CPG3186",
"CPG3210",
"CPG3194",
"CPG3277",
"CPG3244",
"CPG5033",
"CPG919",
"CPG3202",
"CPG5157",
"CPG4846",
"CPG3251",
"CPG4861",
"CPG5140",
"CPG5173",
"CPG4853",
"CPG3228",
"CPG3269",
"CPG5165",
"CPG5041",
"CPG5017",
"CPG5132",
"CPG5058",
"CPG5009",
"CPG5025",
"CPG3236",
"CPG3285",
"CPG3186",
"CPG3210",
"CPG3277",
"CPG3194",
"CPG3202",
"CPG919",
"CPG3244",
"CPG5157",
"CPG3251",
"CPG5033",
"CPG4846",
"CPG3269",
"CPG4861",
"CPG5173",
"CPG5140",
"CPG5165",
"CPG5041",
"CPG4853",
"CPG3228",
"CPG5017",
"CPG5132",
"CPG5009",
"CPG5058",
"CPG5025",
"CPG3236",
"CPG3285",
"CPG3186",
"CPG3277",
"CPG3210",
"CPG3194",
"CPG3244",
"CPG919",
"CPG3202",
"CPG5033",
"CPG3251",
"CPG5157",
"CPG4846",
"CPG4861",
"CPG5140",
"CPG5173",
"CPG3228",
"CPG4853",
"CPG5041",
"CPG3269",
"CPG5165",
"CPG5132",
"CPG5017",
"CPG5058",
"CPG5009",
"CPG5025",
"CPG7021",
"CPG6262",
"CPG8946",
"CPG8847",
"CPG6015",
"CPG3160",
"CPG6585",
"CPG8771",
"CPG3137",
"CPG6742",
"CPG2410",
"CPG8821",
"CPG6577",
"CPG10033",
"CPG8813",
"CPG3129",
"CPG893",
"CPG869",
"CPG3152",
"CPG8953",
"CPG6973",
"CPG11239",
"CPG6593",
"CPG6320",
"CPG8763",
"CPG11247",
"CPG877",
"CPG10090",
"CPG1438",
"CPG3178",
"CPG6270",
"CPG1420",
"CPG6288",
"CPG1495",
"CPG5363",
"CPG5793",
"CPG6999",
"CPG3145",
"CPG851",
"CPG10082",
"CPG1537",
"CPG8789",
"CPG11296",
"CPG11288",
"CPG10058",
"CPG1511",
"CPG8839",
"CPG8755",
"CPG1503",
"CPG10116",
"CPG5355",
"CPG11304",
"CPG7088",
"CPG5371",
"CPG4226",
"CPG1677",
"CPG1354",
"CPG11254",
"CPG1479",
"CPG10132",
"CPG2253",
"CPG5330",
"CPG1669",
"CPG3418",
"CPG1644",
"CPG6569",
"CPG5348",
"CPG1461",
"CPG3400",
"CPG7013",
"CPG11270",
"CPG10140",
"CPG885",
"CPG4358",
"CPG5314",
"CPG4085",
"CPG1487",
"CPG10066",
"CPG3426",
"CPG901",
"CPG8797",
"CPG7047",
"CPG10108",
"CPG6924",
"CPG1362",
"CPG6551",
"CPG1446",
"CPG1370",
"CPG4341",
"CPG5322",
"CPG6916",
"CPG1412",
"CPG1115",
"CPG10074",
"CPG10124",
"CPG6932",
"CPG7005",
"CPG6981",
"CPG1529",
"CPG7039",
"CPG4366",
"CPG7021",
"CPG8847",
"CPG3160",
"CPG6262",
"CPG1487",
"CPG8946",
"CPG8821",
"CPG8813",
"CPG3137",
"CPG3178",
"CPG8771",
"CPG869",
"CPG1420",
"CPG6015",
"CPG6577",
"CPG2410",
"CPG10033",
"CPG6270",
"CPG11296",
"CPG3129",
"CPG3152",
"CPG11239",
"CPG6585",
"CPG6742",
"CPG6999",
"CPG11247",
"CPG8953",
"CPG6593",
"CPG8789",
"CPG893",
"CPG10058",
"CPG1503",
"CPG6288",
"CPG4226",
"CPG877",
"CPG8763",
"CPG1511",
"CPG5793",
"CPG10116",
"CPG11288",
"CPG6320",
"CPG1438",
"CPG851",
"CPG10082",
"CPG3145",
"CPG8755",
"CPG7088",
"CPG11254",
"CPG10140",
"CPG6973",
"CPG1537",
"CPG10090",
"CPG1495",
"CPG1677",
"CPG1461",
"CPG8839",
"CPG11304",
"CPG5363",
"CPG901",
"CPG3418",
"CPG10132",
"CPG6569",
"CPG5355",
"CPG5330",
"CPG5314",
"CPG1354",
"CPG11270",
"CPG10108",
"CPG5371",
"CPG8797",
"CPG1479",
"CPG1644",
"CPG4341",
"CPG2253",
"CPG4358",
"CPG1669",
"CPG4085",
"CPG3400",
"CPG1370",
"CPG3426",
"CPG7005",
"CPG5348",
"CPG1446",
"CPG885",
"CPG6924",
"CPG6551",
"CPG7013",
"CPG10066",
"CPG7047",
"CPG6932",
"CPG10124",
"CPG1362",
"CPG10074",
"CPG6916",
"CPG1115",
"CPG1529",
"CPG5322",
"CPG1412",
"CPG7039",
"CPG6981",
"CPG4366",
"CPG2410",
"CPG1420",
"CPG11296",
"CPG1537",
"CPG1511",
"CPG7088",
"CPG1438",
"CPG1495",
"CPG1503",
"CPG11288",
"CPG1461",
"CPG1354",
"CPG1446",
"CPG11304",
"CPG4085",
"CPG11270",
"CPG1479",
"CPG3418",
"CPG1487",
"CPG3400",
"CPG3426",
"CPG2253",
"CPG1370",
"CPG1362",
"CPG1412",
"CPG1115",
"CPG1529",
"CPG2410",
"CPG1420",
"CPG11296",
"CPG1537",
"CPG1511",
"CPG1495",
"CPG11288",
"CPG1438",
"CPG1503",
"CPG7088",
"CPG1461",
"CPG11304",
"CPG3418",
"CPG1354",
"CPG1446",
"CPG4085",
"CPG1479",
"CPG11270",
"CPG3400",
"CPG1370",
"CPG3426",
"CPG2253",
"CPG1487",
"CPG1362",
"CPG1412",
"CPG1529",
"CPG1115",
"CPG2410",
"CPG1420",
"CPG11296",
"CPG1438",
"CPG11288",
"CPG1511",
"CPG1537",
"CPG1495",
"CPG7088",
"CPG1503",
"CPG1461",
"CPG3418",
"CPG1354",
"CPG11270",
"CPG1479",
"CPG11304",
"CPG1446",
"CPG1487",
"CPG4085",
"CPG1370",
"CPG3400",
"CPG3426",
"CPG2253",
"CPG1362",
"CPG1412",
"CPG1115",
"CPG1529",
"CPG3160",
"CPG3137",
"CPG869",
"CPG3129",
"CPG3152",
"CPG3178",
"CPG877",
"CPG3145",
"CPG4226",
"CPG851",
"CPG5363",
"CPG5314",
"CPG1677",
"CPG5355",
"CPG1644",
"CPG4358",
"CPG901",
"CPG5330",
"CPG5371",
"CPG885",
"CPG4341",
"CPG5348",
"CPG1669",
"CPG5322",
"CPG893",
"CPG4366",
"CPG3160",
"CPG3137",
"CPG869",
"CPG3178",
"CPG3129",
"CPG877",
"CPG3145",
"CPG3152",
"CPG851",
"CPG4226",
"CPG1644",
"CPG5363",
"CPG901",
"CPG1677",
"CPG5314",
"CPG5371",
"CPG4358",
"CPG5330",
"CPG5355",
"CPG885",
"CPG5348",
"CPG1669",
"CPG4341",
"CPG5322",
"CPG893",
"CPG4366",
"CPG3160",
"CPG3137",
"CPG3129",
"CPG869",
"CPG3152",
"CPG877",
"CPG3178",
"CPG3145",
"CPG4226",
"CPG851",
"CPG5363",
"CPG5355",
"CPG1644",
"CPG893",
"CPG1677",
"CPG901",
"CPG5314",
"CPG5330",
"CPG5371",
"CPG4358",
"CPG885",
"CPG4341",
"CPG1669",
"CPG5348",
"CPG5322",
"CPG4366",
"CPG8946",
"CPG8847",
"CPG8771",
"CPG8813",
"CPG8821",
"CPG8953",
"CPG8763",
"CPG8789",
"CPG10033",
"CPG10116",
"CPG10082",
"CPG8755",
"CPG10090",
"CPG8839",
"CPG10058",
"CPG10132",
"CPG10140",
"CPG10108",
"CPG8797",
"CPG10066",
"CPG10124",
"CPG10074",
"CPG8946",
"CPG8847",
"CPG8771",
"CPG8821",
"CPG8813",
"CPG8763",
"CPG10033",
"CPG8789",
"CPG8953",
"CPG10082",
"CPG10090",
"CPG10116",
"CPG8755",
"CPG8839",
"CPG10058",
"CPG10132",
"CPG8797",
"CPG10140",
"CPG10108",
"CPG10066",
"CPG10124",
"CPG10074",
"CPG8847",
"CPG8755",
"CPG8946",
"CPG8771",
"CPG8813",
"CPG8821",
"CPG8953",
"CPG8789",
"CPG10033",
"CPG8763",
"CPG10082",
"CPG10116",
"CPG8839",
"CPG10090",
"CPG10058",
"CPG10140",
"CPG8797",
"CPG10132",
"CPG10108",
"CPG10066",
"CPG10124",
"CPG10074",
"CPG7021",
"CPG6262",
"CPG6585",
"CPG6015",
"CPG11239",
"CPG11247",
"CPG6577",
"CPG6999",
"CPG6270",
"CPG6742",
"CPG6320",
"CPG6288",
"CPG6593",
"CPG6973",
"CPG5793",
"CPG11254",
"CPG6569",
"CPG7013",
"CPG6924",
"CPG6551",
"CPG6916",
"CPG7047",
"CPG6932",
"CPG7005",
"CPG7039",
"CPG6981",
"CPG7021",
"CPG6015",
"CPG6262",
"CPG6585",
"CPG6742",
"CPG11247",
"CPG6577",
"CPG6270",
"CPG11239",
"CPG6999",
"CPG6593",
"CPG6320",
"CPG6288",
"CPG5793",
"CPG6973",
"CPG11254",
"CPG6569",
"CPG7013",
"CPG7047",
"CPG6551",
"CPG6924",
"CPG7005",
"CPG6916",
"CPG6932",
"CPG7039",
"CPG6981",
"CPG7021",
"CPG6262",
"CPG6585",
"CPG6015",
"CPG11247",
"CPG6577",
"CPG11239",
"CPG6742",
"CPG6593",
"CPG6270",
"CPG6320",
"CPG6999",
"CPG6288",
"CPG6973",
"CPG5793",
"CPG11254",
"CPG6569",
"CPG7047",
"CPG7013",
"CPG6551",
"CPG6924",
"CPG6932",
"CPG6916",
"CPG7005",
"CPG6981",
"CPG7039",
"CPG2824",
"CPG2832",
"CPG2469",
"CPG6387",
"CPG6734",
"CPG2949",
"CPG2907",
"CPG6692",
"CPG3970",
"CPG2881",
"CPG6486",
"CPG6627",
"CPG6445",
"CPG9290",
"CPG6429",
"CPG6965",
"CPG6825",
"CPG1701",
"CPG2477",
"CPG737",
"CPG6452",
"CPG6478",
"CPG2857",
"CPG6791",
"CPG6437",
"CPG6775",
"CPG3988",
"CPG6619",
"CPG6833",
"CPG6676",
"CPG6668",
"CPG6395",
"CPG810",
"CPG6809",
"CPG745",
"CPG2840",
"CPG9274",
"CPG1024",
"CPG976",
"CPG6403",
"CPG6718",
"CPG6817",
"CPG6411",
"CPG1040",
"CPG828",
"CPG6866",
"CPG6635",
"CPG9316",
"CPG6379",
"CPG992",
"CPG2915",
"CPG6684",
"CPG968",
"CPG9266",
"CPG802",
"CPG5587",
"CPG6353",
"CPG9332",
"CPG9282",
"CPG4051",
"CPG2873",
"CPG6460",
"CPG6650",
"CPG6643",
"CPG1099",
"CPG6874",
"CPG6726",
"CPG6700",
"CPG2899",
"CPG9308",
"CPG1032",
"CPG1081",
"CPG9324",
"CPG9704",
"CPG9340",
"CPG4028",
"CPG6361",
"CPG5934",
"CPG2485",
"CPG7120",
"CPG2501",
"CPG6338",
"CPG6502",
"CPG9357",
"CPG9365",
"CPG5843",
"CPG1057",
"CPG6346",
"CPG9258",
"CPG6494",
"CPG5918",
"CPG5181",
"CPG6783",
"CPG1693",
"CPG1016",
"CPG1073",
"CPG9373",
"CPG5926",
"CPG5827",
"CPG5868",
"CPG1065",
"CPG5801",
"CPG1784",
"CPG2493",
"CPG5900",
"CPG6858",
"CPG6601",
"CPG6841",
"CPG5819",
"CPG5835",
"CPG5850",
"CPG2824",
"CPG2832",
"CPG2907",
"CPG2469",
"CPG2949",
"CPG6387",
"CPG6734",
"CPG6965",
"CPG6429",
"CPG6478",
"CPG2857",
"CPG6452",
"CPG6692",
"CPG6833",
"CPG6676",
"CPG6486",
"CPG2915",
"CPG6627",
"CPG3970",
"CPG2477",
"CPG9290",
"CPG6791",
"CPG2881",
"CPG2873",
"CPG968",
"CPG6866",
"CPG9274",
"CPG6809",
"CPG1024",
"CPG6700",
"CPG2840",
"CPG745",
"CPG6445",
"CPG6403",
"CPG6379",
"CPG976",
"CPG6718",
"CPG6825",
"CPG3988",
"CPG6668",
"CPG810",
"CPG828",
"CPG6619",
"CPG6643",
"CPG1701",
"CPG1040",
"CPG737",
"CPG9332",
"CPG9266",
"CPG4051",
"CPG992",
"CPG6395",
"CPG9316",
"CPG6635",
"CPG6775",
"CPG802",
"CPG6817",
"CPG6353",
"CPG6726",
"CPG9340",
"CPG1081",
"CPG9282",
"CPG6437",
"CPG6502",
"CPG6411",
"CPG4028",
"CPG6650",
"CPG5587",
"CPG6874",
"CPG2899",
"CPG6460",
"CPG1032",
"CPG9308",
"CPG6346",
"CPG5934",
"CPG6361",
"CPG9357",
"CPG7120",
"CPG1099",
"CPG9704",
"CPG6684",
"CPG9258",
"CPG2485",
"CPG6338",
"CPG5918",
"CPG6783",
"CPG1016",
"CPG9365",
"CPG6494",
"CPG1057",
"CPG2501",
"CPG1784",
"CPG5868",
"CPG5181",
"CPG5843",
"CPG9324",
"CPG1065",
"CPG5827",
"CPG1693",
"CPG6601",
"CPG5926",
"CPG1073",
"CPG5801",
"CPG9373",
"CPG5900",
"CPG6858",
"CPG2493",
"CPG5850",
"CPG6841",
"CPG5835",
"CPG5819",
"CPG6734",
"CPG6627",
"CPG6692",
"CPG9290",
"CPG6619",
"CPG6676",
"CPG9274",
"CPG6668",
"CPG9266",
"CPG6684",
"CPG6635",
"CPG9316",
"CPG6718",
"CPG6643",
"CPG6700",
"CPG9332",
"CPG6650",
"CPG6726",
"CPG9282",
"CPG9340",
"CPG9308",
"CPG9704",
"CPG9357",
"CPG9258",
"CPG9365",
"CPG9373",
"CPG9324",
"CPG6601",
"CPG6734",
"CPG6692",
"CPG6627",
"CPG9290",
"CPG6676",
"CPG6619",
"CPG9274",
"CPG6668",
"CPG9316",
"CPG6718",
"CPG6684",
"CPG6700",
"CPG9266",
"CPG6635",
"CPG6643",
"CPG6726",
"CPG9282",
"CPG9332",
"CPG9340",
"CPG6650",
"CPG9308",
"CPG9704",
"CPG9357",
"CPG9365",
"CPG9258",
"CPG9373",
"CPG9324",
"CPG6601",
"CPG6734",
"CPG6692",
"CPG6627",
"CPG9290",
"CPG6676",
"CPG6619",
"CPG9274",
"CPG6718",
"CPG6668",
"CPG9316",
"CPG6684",
"CPG9266",
"CPG6700",
"CPG6635",
"CPG9332",
"CPG6643",
"CPG6650",
"CPG9282",
"CPG9340",
"CPG6726",
"CPG9308",
"CPG9704",
"CPG9365",
"CPG9258",
"CPG9357",
"CPG9373",
"CPG9324",
"CPG6601",
"CPG6387",
"CPG2469",
"CPG6429",
"CPG6965",
"CPG745",
"CPG2477",
"CPG6775",
"CPG968",
"CPG6395",
"CPG1024",
"CPG6403",
"CPG737",
"CPG976",
"CPG1040",
"CPG6437",
"CPG6353",
"CPG6411",
"CPG992",
"CPG6379",
"CPG6361",
"CPG6338",
"CPG1032",
"CPG6346",
"CPG6783",
"CPG2485",
"CPG1016",
"CPG2501",
"CPG2493",
"CPG6387",
"CPG2469",
"CPG6429",
"CPG6965",
"CPG2477",
"CPG6775",
"CPG745",
"CPG6403",
"CPG976",
"CPG968",
"CPG6395",
"CPG737",
"CPG1040",
"CPG6379",
"CPG1024",
"CPG6411",
"CPG992",
"CPG6353",
"CPG6437",
"CPG1032",
"CPG6361",
"CPG6346",
"CPG6338",
"CPG2485",
"CPG6783",
"CPG1016",
"CPG2501",
"CPG2493",
"CPG6387",
"CPG2469",
"CPG6965",
"CPG6429",
"CPG745",
"CPG2477",
"CPG6775",
"CPG737",
"CPG6395",
"CPG968",
"CPG1024",
"CPG6403",
"CPG976",
"CPG1040",
"CPG6411",
"CPG6353",
"CPG992",
"CPG6379",
"CPG6437",
"CPG1032",
"CPG2485",
"CPG6361",
"CPG6338",
"CPG6783",
"CPG6346",
"CPG1016",
"CPG2501",
"CPG2493",
"CPG4051",
"CPG1701",
"CPG4028",
"CPG1081",
"CPG5587",
"CPG6502",
"CPG5934",
"CPG1099",
"CPG7120",
"CPG5843",
"CPG1057",
"CPG5918",
"CPG5926",
"CPG1784",
"CPG5868",
"CPG5181",
"CPG6494",
"CPG1065",
"CPG5827",
"CPG1693",
"CPG1073",
"CPG5801",
"CPG5900",
"CPG6858",
"CPG5850",
"CPG6841",
"CPG5835",
"CPG5819",
"CPG1701",
"CPG4051",
"CPG6502",
"CPG5587",
"CPG1081",
"CPG4028",
"CPG5934",
"CPG1099",
"CPG5843",
"CPG7120",
"CPG1057",
"CPG5918",
"CPG5926",
"CPG1784",
"CPG6494",
"CPG5181",
"CPG5827",
"CPG1693",
"CPG5868",
"CPG1073",
"CPG5801",
"CPG1065",
"CPG6858",
"CPG5900",
"CPG5850",
"CPG6841",
"CPG5835",
"CPG5819",
"CPG1701",
"CPG4051",
"CPG6502",
"CPG4028",
"CPG1081",
"CPG5934",
"CPG5587",
"CPG1099",
"CPG6494",
"CPG5843",
"CPG7120",
"CPG1784",
"CPG1057",
"CPG5918",
"CPG5926",
"CPG5181",
"CPG5868",
"CPG5827",
"CPG1065",
"CPG1693",
"CPG1073",
"CPG5801",
"CPG5900",
"CPG6858",
"CPG6841",
"CPG5850",
"CPG5835",
"CPG5819",
"CPG2824",
"CPG2832",
"CPG2907",
"CPG2949",
"CPG6486",
"CPG6452",
"CPG3970",
"CPG2857",
"CPG6478",
"CPG2915",
"CPG6445",
"CPG6833",
"CPG6791",
"CPG2873",
"CPG6866",
"CPG828",
"CPG2881",
"CPG2840",
"CPG6825",
"CPG3988",
"CPG6817",
"CPG810",
"CPG802",
"CPG2899",
"CPG6460",
"CPG6809",
"CPG6874",
"CPG2824",
"CPG2832",
"CPG2907",
"CPG2949",
"CPG6486",
"CPG3970",
"CPG2857",
"CPG6445",
"CPG6452",
"CPG6833",
"CPG2915",
"CPG6809",
"CPG6791",
"CPG6478",
"CPG2881",
"CPG6866",
"CPG6825",
"CPG6817",
"CPG810",
"CPG2840",
"CPG828",
"CPG3988",
"CPG2873",
"CPG802",
"CPG6460",
"CPG6874",
"CPG2899",
"CPG2824",
"CPG2832",
"CPG2907",
"CPG2949",
"CPG6486",
"CPG3970",
"CPG6817",
"CPG2915",
"CPG6445",
"CPG2857",
"CPG6452",
"CPG6478",
"CPG6825",
"CPG6833",
"CPG6809",
"CPG3988",
"CPG828",
"CPG2840",
"CPG6791",
"CPG2873",
"CPG6866",
"CPG810",
"CPG2881",
"CPG802",
"CPG6460",
"CPG6874",
"CPG2899",
"CPG4184",
"CPG4168",
"CPG4192",
"CPG4176",
"CPG7435",
"CPG4200",
"CPG7419",
"CPG1628",
"CPG7377",
"CPG7401",
"CPG7450",
"CPG4929",
"CPG1610",
"CPG10744",
"CPG4887",
"CPG1206",
"CPG1214",
"CPG10819",
"CPG7393",
"CPG2386",
"CPG7385",
"CPG3293",
"CPG1271",
"CPG7351",
"CPG1263",
"CPG4895",
"CPG2444",
"CPG4143",
"CPG10751",
"CPG2519",
"CPG7427",
"CPG7443",
"CPG10736",
"CPG1305",
"CPG2378",
"CPG9563",
"CPG4135",
"CPG2113",
"CPG4911",
"CPG7344",
"CPG2402",
"CPG1602",
"CPG10827",
"CPG4903",
"CPG9621",
"CPG8649",
"CPG4879",
"CPG11262",
"CPG10793",
"CPG1297",
"CPG2212",
"CPG2204",
"CPG7369",
"CPG10801",
"CPG10447",
"CPG10785",
"CPG9613",
"CPG10421",
"CPG10405",
"CPG2121",
"CPG2394",
"CPG10777",
"CPG2097",
"CPG10850",
"CPG10769",
"CPG760",
"CPG10892",
"CPG10868",
"CPG2063",
"CPG9555",
"CPG2527",
"CPG2154",
"CPG9548",
"CPG9043",
"CPG3392",
"CPG10454",
"CPG1222",
"CPG9647",
"CPG2238",
"CPG2436",
"CPG2220",
"CPG9605",
"CPG10835",
"CPG2147",
"CPG10470",
"CPG10876",
"CPG10843",
"CPG752",
"CPG10488",
"CPG9571",
"CPG2451",
"CPG2105",
"CPG2139",
"CPG8631",
"CPG2071",
"CPG1883",
"CPG10413",
"CPG10355",
"CPG2089",
"CPG9589",
"CPG9597",
"CPG8623",
"CPG10884",
"CPG9639",
"CPG10439",
"CPG4168",
"CPG4184",
"CPG4200",
"CPG4176",
"CPG4192",
"CPG7419",
"CPG7435",
"CPG1628",
"CPG7393",
"CPG4143",
"CPG4895",
"CPG7401",
"CPG7377",
"CPG4929",
"CPG7450",
"CPG1305",
"CPG7385",
"CPG2519",
"CPG7351",
"CPG3293",
"CPG4879",
"CPG4887",
"CPG1214",
"CPG10736",
"CPG1206",
"CPG1610",
"CPG1271",
"CPG2386",
"CPG10744",
"CPG7427",
"CPG7344",
"CPG4911",
"CPG9563",
"CPG10751",
"CPG2378",
"CPG10819",
"CPG2444",
"CPG7443",
"CPG2402",
"CPG10405",
"CPG4135",
"CPG1263",
"CPG9621",
"CPG4903",
"CPG1602",
"CPG10850",
"CPG10793",
"CPG10769",
"CPG10827",
"CPG2527",
"CPG9613",
"CPG2113",
"CPG3392",
"CPG760",
"CPG10777",
"CPG10488",
"CPG10843",
"CPG2097",
"CPG10447",
"CPG2212",
"CPG10785",
"CPG2154",
"CPG2204",
"CPG7369",
"CPG1297",
"CPG2147",
"CPG10421",
"CPG2394",
"CPG10801",
"CPG11262",
"CPG9555",
"CPG2220",
"CPG2436",
"CPG8649",
"CPG2063",
"CPG9647",
"CPG2121",
"CPG9605",
"CPG10470",
"CPG9043",
"CPG10835",
"CPG2238",
"CPG10454",
"CPG2139",
"CPG10876",
"CPG10868",
"CPG2071",
"CPG752",
"CPG10892",
"CPG8623",
"CPG1883",
"CPG2089",
"CPG9548",
"CPG10355",
"CPG1222",
"CPG9571",
"CPG2451",
"CPG2105",
"CPG9597",
"CPG8631",
"CPG10413",
"CPG10884",
"CPG9589",
"CPG9639",
"CPG10439",
"CPG4168",
"CPG4192",
"CPG4184",
"CPG4176",
"CPG4200",
"CPG7435",
"CPG7419",
"CPG7401",
"CPG7377",
"CPG7393",
"CPG4895",
"CPG7351",
"CPG4887",
"CPG7450",
"CPG7385",
"CPG4929",
"CPG4143",
"CPG7427",
"CPG4135",
"CPG4911",
"CPG4903",
"CPG4879",
"CPG7344",
"CPG7443",
"CPG7369",
"CPG4168",
"CPG4192",
"CPG4184",
"CPG4176",
"CPG4200",
"CPG7419",
"CPG7435",
"CPG7351",
"CPG4895",
"CPG7393",
"CPG7401",
"CPG7377",
"CPG4887",
"CPG4929",
"CPG4143",
"CPG7450",
"CPG7385",
"CPG4879",
"CPG4911",
"CPG7427",
"CPG4903",
"CPG7443",
"CPG4135",
"CPG7344",
"CPG7369",
"CPG4168",
"CPG4192",
"CPG4184",
"CPG4200",
"CPG4176",
"CPG7419",
"CPG7435",
"CPG7401",
"CPG7377",
"CPG4895",
"CPG7393",
"CPG7351",
"CPG4887",
"CPG4929",
"CPG7450",
"CPG4143",
"CPG7385",
"CPG4879",
"CPG4135",
"CPG7427",
"CPG4911",
"CPG4903",
"CPG7443",
"CPG7344",
"CPG7369",
"CPG2386",
"CPG2378",
"CPG2113",
"CPG2402",
"CPG10447",
"CPG10405",
"CPG8649",
"CPG10488",
"CPG10470",
"CPG2154",
"CPG2063",
"CPG2147",
"CPG10421",
"CPG2121",
"CPG9043",
"CPG2097",
"CPG10454",
"CPG2139",
"CPG2394",
"CPG8631",
"CPG2071",
"CPG2105",
"CPG2089",
"CPG8623",
"CPG10413",
"CPG10355",
"CPG10439",
"CPG2386",
"CPG2378",
"CPG2113",
"CPG2402",
"CPG10447",
"CPG10405",
"CPG8649",
"CPG2121",
"CPG2154",
"CPG9043",
"CPG10470",
"CPG2063",
"CPG2147",
"CPG10421",
"CPG10488",
"CPG10454",
"CPG2394",
"CPG2139",
"CPG2097",
"CPG8631",
"CPG2105",
"CPG2071",
"CPG10355",
"CPG8623",
"CPG2089",
"CPG10413",
"CPG10439",
"CPG2386",
"CPG2378",
"CPG2113",
"CPG2402",
"CPG10405",
"CPG10447",
"CPG8649",
"CPG10470",
"CPG2147",
"CPG2154",
"CPG10454",
"CPG9043",
"CPG10488",
"CPG10421",
"CPG2063",
"CPG2394",
"CPG2097",
"CPG2121",
"CPG8631",
"CPG2105",
"CPG2139",
"CPG2071",
"CPG8623",
"CPG2089",
"CPG10413",
"CPG10355",
"CPG10439",
"CPG1628",
"CPG1610",
"CPG1206",
"CPG1305",
"CPG1214",
"CPG2519",
"CPG3293",
"CPG2444",
"CPG1271",
"CPG1263",
"CPG2212",
"CPG760",
"CPG11262",
"CPG1602",
"CPG1297",
"CPG2527",
"CPG2238",
"CPG2204",
"CPG2220",
"CPG1222",
"CPG3392",
"CPG2436",
"CPG2451",
"CPG752",
"CPG1883",
"CPG1628",
"CPG1214",
"CPG1206",
"CPG1610",
"CPG1305",
"CPG2519",
"CPG3293",
"CPG1271",
"CPG2444",
"CPG1263",
"CPG1297",
"CPG2212",
"CPG1602",
"CPG2238",
"CPG760",
"CPG11262",
"CPG2527",
"CPG2204",
"CPG2436",
"CPG2220",
"CPG3392",
"CPG1883",
"CPG2451",
"CPG1222",
"CPG752",
"CPG1628",
"CPG1206",
"CPG1610",
"CPG1214",
"CPG1305",
"CPG3293",
"CPG2519",
"CPG1271",
"CPG2444",
"CPG1263",
"CPG1297",
"CPG1602",
"CPG11262",
"CPG2212",
"CPG760",
"CPG2238",
"CPG2527",
"CPG2204",
"CPG2436",
"CPG752",
"CPG1222",
"CPG2220",
"CPG3392",
"CPG2451",
"CPG1883",
"CPG10744",
"CPG10736",
"CPG10819",
"CPG10751",
"CPG9563",
"CPG10793",
"CPG9613",
"CPG10801",
"CPG9621",
"CPG10892",
"CPG10827",
"CPG10769",
"CPG10850",
"CPG10835",
"CPG10777",
"CPG10868",
"CPG9555",
"CPG10785",
"CPG9548",
"CPG9647",
"CPG10843",
"CPG10876",
"CPG9605",
"CPG9571",
"CPG9639",
"CPG9597",
"CPG9589",
"CPG10884",
"CPG10744",
"CPG10736",
"CPG10819",
"CPG9563",
"CPG10751",
"CPG10793",
"CPG9613",
"CPG10850",
"CPG10835",
"CPG10769",
"CPG9621",
"CPG10801",
"CPG10892",
"CPG10777",
"CPG10827",
"CPG10785",
"CPG9555",
"CPG10868",
"CPG9647",
"CPG10843",
"CPG9548",
"CPG9605",
"CPG9639",
"CPG10876",
"CPG9571",
"CPG9597",
"CPG10884",
"CPG9589",
"CPG10744",
"CPG10736",
"CPG10751",
"CPG10819",
"CPG10793",
"CPG9563",
"CPG9613",
"CPG10801",
"CPG10769",
"CPG10892",
"CPG10827",
"CPG9621",
"CPG10850",
"CPG10777",
"CPG9555",
"CPG10835",
"CPG10785",
"CPG10868",
"CPG9548",
"CPG10843",
"CPG9605",
"CPG9647",
"CPG10876",
"CPG9571",
"CPG9597",
"CPG9589",
"CPG10884",
"CPG9639",
"CPG10173",
"CPG10967",
"CPG9845",
"CPG9175",
"CPG10207",
"CPG9480",
"CPG7880",
"CPG9167",
"CPG9811",
"CPG7740",
"CPG10975",
"CPG7690",
"CPG10215",
"CPG7724",
"CPG7922",
"CPG9431",
"CPG3749",
"CPG6023",
"CPG10199",
"CPG7955",
"CPG5637",
"CPG9423",
"CPG9795",
"CPG9134",
"CPG5652",
"CPG9852",
"CPG10959",
"CPG3798",
"CPG11171",
"CPG9142",
"CPG5678",
"CPG10272",
"CPG9159",
"CPG10181",
"CPG10652",
"CPG7708",
"CPG7765",
"CPG10926",
"CPG10942",
"CPG9894",
"CPG7666",
"CPG1776",
"CPG8557",
"CPG9126",
"CPG10264",
"CPG10702",
"CPG7898",
"CPG5629",
"CPG7757",
"CPG10678",
"CPG9829",
"CPG8540",
"CPG3764",
"CPG10561",
"CPG8920",
"CPG7732",
"CPG6049",
"CPG9787",
"CPG7914",
"CPG9878",
"CPG10686",
"CPG10256",
"CPG3780",
"CPG9530",
"CPG9860",
"CPG3756",
"CPG8664",
"CPG8672",
"CPG10231",
"CPG8524",
"CPG6031",
"CPG8565",
"CPG10645",
"CPG10546",
"CPG7773",
"CPG10934",
"CPG3772",
"CPG8748",
"CPG8490",
"CPG8474",
"CPG3731",
"CPG10710",
"CPG8532",
"CPG9522",
"CPG7906",
"CPG9506",
"CPG7781",
"CPG9514",
"CPG6007",
"CPG9464",
"CPG8730",
"CPG8482",
"CPG8516",
"CPG6080",
"CPG8466",
"CPG10553",
"CPG10223",
"CPG10280",
"CPG5645",
"CPG10694",
"CPG8029",
"CPG5660",
"CPG8508",
"CPG3723",
"CPG10173",
"CPG10199",
"CPG10207",
"CPG10967",
"CPG7955",
"CPG9845",
"CPG9175",
"CPG7690",
"CPG9167",
"CPG7740",
"CPG9480",
"CPG6023",
"CPG10975",
"CPG7880",
"CPG7922",
"CPG5637",
"CPG9795",
"CPG9811",
"CPG9423",
"CPG7724",
"CPG3798",
"CPG7666",
"CPG9431",
"CPG10272",
"CPG9134",
"CPG3749",
"CPG9126",
"CPG7708",
"CPG5652",
"CPG9159",
"CPG9142",
"CPG10959",
"CPG7765",
"CPG11171",
"CPG10215",
"CPG10942",
"CPG10264",
"CPG9894",
"CPG8920",
"CPG8565",
"CPG7898",
"CPG5678",
"CPG10181",
"CPG7757",
"CPG10652",
"CPG9852",
"CPG5629",
"CPG3731",
"CPG1776",
"CPG8557",
"CPG8540",
"CPG8524",
"CPG10934",
"CPG3756",
"CPG10926",
"CPG3764",
"CPG10702",
"CPG7732",
"CPG10561",
"CPG3780",
"CPG10686",
"CPG10678",
"CPG7773",
"CPG9829",
"CPG6031",
"CPG8664",
"CPG7914",
"CPG10645",
"CPG9530",
"CPG3772",
"CPG10256",
"CPG9787",
"CPG8748",
"CPG7781",
"CPG8482",
"CPG9878",
"CPG9860",
"CPG10546",
"CPG6007",
"CPG8490",
"CPG10553",
"CPG6049",
"CPG9514",
"CPG9506",
"CPG9522",
"CPG9464",
"CPG10231",
"CPG8730",
"CPG8532",
"CPG8516",
"CPG10710",
"CPG10280",
"CPG8672",
"CPG7906",
"CPG8474",
"CPG6080",
"CPG10223",
"CPG5660",
"CPG8466",
"CPG8508",
"CPG10694",
"CPG5645",
"CPG8029",
"CPG3723",
"CPG10967",
"CPG10975",
"CPG6023",
"CPG3798",
"CPG5637",
"CPG5652",
"CPG10959",
"CPG10942",
"CPG3749",
"CPG11171",
"CPG1776",
"CPG5678",
"CPG10926",
"CPG3780",
"CPG5629",
"CPG3756",
"CPG3764",
"CPG6049",
"CPG6031",
"CPG10934",
"CPG3731",
"CPG3772",
"CPG6007",
"CPG6080",
"CPG5645",
"CPG5660",
"CPG3723",
"CPG10967",
"CPG5637",
"CPG10975",
"CPG6023",
"CPG3798",
"CPG3749",
"CPG10942",
"CPG11171",
"CPG5652",
"CPG10959",
"CPG1776",
"CPG5678",
"CPG3756",
"CPG10926",
"CPG3764",
"CPG5629",
"CPG3780",
"CPG6031",
"CPG10934",
"CPG3772",
"CPG3731",
"CPG6049",
"CPG6007",
"CPG6080",
"CPG5645",
"CPG5660",
"CPG3723",
"CPG10967",
"CPG5637",
"CPG10975",
"CPG10959",
"CPG3798",
"CPG10942",
"CPG6023",
"CPG5652",
"CPG11171",
"CPG3749",
"CPG1776",
"CPG5678",
"CPG3764",
"CPG3756",
"CPG5629",
"CPG3780",
"CPG10926",
"CPG10934",
"CPG3772",
"CPG6031",
"CPG3731",
"CPG6007",
"CPG6049",
"CPG6080",
"CPG5660",
"CPG5645",
"CPG3723",
"CPG10173",
"CPG10199",
"CPG9845",
"CPG10207",
"CPG7922",
"CPG7880",
"CPG7690",
"CPG7724",
"CPG9811",
"CPG7740",
"CPG7955",
"CPG9795",
"CPG7898",
"CPG7708",
"CPG9852",
"CPG9894",
"CPG9829",
"CPG7732",
"CPG7914",
"CPG9878",
"CPG9787",
"CPG9860",
"CPG7906",
"CPG10173",
"CPG10207",
"CPG7880",
"CPG10199",
"CPG9845",
"CPG7922",
"CPG7955",
"CPG9811",
"CPG7690",
"CPG7740",
"CPG7724",
"CPG9795",
"CPG7708",
"CPG9852",
"CPG7898",
"CPG9894",
"CPG7732",
"CPG9829",
"CPG9878",
"CPG9787",
"CPG7914",
"CPG9860",
"CPG7906",
"CPG10173",
"CPG10207",
"CPG10199",
"CPG7880",
"CPG7922",
"CPG7690",
"CPG9845",
"CPG9811",
"CPG7740",
"CPG7955",
"CPG7724",
"CPG9795",
"CPG7708",
"CPG7898",
"CPG9852",
"CPG9894",
"CPG7732",
"CPG9787",
"CPG9829",
"CPG7914",
"CPG9878",
"CPG9860",
"CPG7906",
"CPG9431",
"CPG9175",
"CPG9142",
"CPG9134",
"CPG9159",
"CPG9423",
"CPG9126",
"CPG9167",
"CPG8557",
"CPG8920",
"CPG8565",
"CPG8540",
"CPG10561",
"CPG8524",
"CPG8748",
"CPG8664",
"CPG8490",
"CPG10546",
"CPG8730",
"CPG8474",
"CPG8482",
"CPG8672",
"CPG8532",
"CPG8516",
"CPG10553",
"CPG8466",
"CPG8029",
"CPG8508",
"CPG9431",
"CPG9175",
"CPG9167",
"CPG9423",
"CPG9134",
"CPG9142",
"CPG9159",
"CPG9126",
"CPG8557",
"CPG8920",
"CPG8565",
"CPG10561",
"CPG8540",
"CPG8524",
"CPG8748",
"CPG10546",
"CPG8664",
"CPG8490",
"CPG8730",
"CPG8672",
"CPG8474",
"CPG8482",
"CPG8532",
"CPG8516",
"CPG10553",
"CPG8466",
"CPG8029",
"CPG8508",
"CPG9175",
"CPG9431",
"CPG9142",
"CPG9423",
"CPG9134",
"CPG9167",
"CPG9159",
"CPG9126",
"CPG10561",
"CPG8565",
"CPG8557",
"CPG8920",
"CPG8540",
"CPG8664",
"CPG8524",
"CPG8730",
"CPG8490",
"CPG10546",
"CPG8748",
"CPG8474",
"CPG8516",
"CPG8672",
"CPG8532",
"CPG8482",
"CPG8466",
"CPG10553",
"CPG8029",
"CPG8508",
"CPG9480",
"CPG10272",
"CPG7666",
"CPG10215",
"CPG7765",
"CPG10181",
"CPG7757",
"CPG10264",
"CPG10652",
"CPG10678",
"CPG10702",
"CPG10686",
"CPG7773",
"CPG10256",
"CPG10645",
"CPG7781",
"CPG9530",
"CPG9506",
"CPG9514",
"CPG10710",
"CPG9522",
"CPG10231",
"CPG10223",
"CPG9464",
"CPG10280",
"CPG10694",
"CPG9480",
"CPG10272",
"CPG10215",
"CPG7765",
"CPG7757",
"CPG7666",
"CPG9514",
"CPG10652",
"CPG10264",
"CPG10181",
"CPG10702",
"CPG10678",
"CPG7773",
"CPG10256",
"CPG10645",
"CPG7781",
"CPG9506",
"CPG9530",
"CPG10686",
"CPG9522",
"CPG10710",
"CPG10231",
"CPG10223",
"CPG9464",
"CPG10280",
"CPG10694",
"CPG9480",
"CPG7666",
"CPG10215",
"CPG10272",
"CPG7765",
"CPG10181",
"CPG10264",
"CPG7757",
"CPG10652",
"CPG10678",
"CPG10702",
"CPG10686",
"CPG7773",
"CPG9506",
"CPG10256",
"CPG10645",
"CPG7781",
"CPG9530",
"CPG9514",
"CPG10710",
"CPG10231",
"CPG9522",
"CPG10223",
"CPG9464",
"CPG10280",
"CPG10694",
"CPG1941",
"CPG1859",
"CPG4689",
"CPG4499",
"CPG1891",
"CPG1834",
"CPG4705",
"CPG11221",
"CPG4952",
"CPG1909",
"CPG1875",
"CPG1925",
"CPG1867",
"CPG1933",
"CPG4473",
"CPG1842",
"CPG4697",
"CPG1917",
"CPG4218",
"CPG1966",
"CPG4721",
"CPG4523",
"CPG1958",
"CPG11213",
"CPG4713",
"CPG4515",
"CPG1974",
"CPG4481",
"CPG1941",
"CPG1859",
"CPG4689",
"CPG1834",
"CPG1891",
"CPG4499",
"CPG1875",
"CPG11221",
"CPG4952",
"CPG4705",
"CPG1925",
"CPG1909",
"CPG1867",
"CPG1933",
"CPG4473",
"CPG4697",
"CPG1842",
"CPG4721",
"CPG1966",
"CPG4218",
"CPG1917",
"CPG4713",
"CPG1958",
"CPG4523",
"CPG11213",
"CPG1974",
"CPG4515",
"CPG4481",
"CPG1941",
"CPG1859",
"CPG4689",
"CPG1834",
"CPG4952",
"CPG1891",
"CPG11221",
"CPG4499",
"CPG1875",
"CPG4705",
"CPG1909",
"CPG1925",
"CPG1867",
"CPG1933",
"CPG4473",
"CPG4697",
"CPG1842",
"CPG1917",
"CPG4721",
"CPG4218",
"CPG1966",
"CPG1958",
"CPG4523",
"CPG11213",
"CPG4713",
"CPG1974",
"CPG4515",
"CPG4481",
"CPG1941",
"CPG4689",
"CPG1859",
"CPG1834",
"CPG1891",
"CPG4499",
"CPG11221",
"CPG4952",
"CPG1933",
"CPG1867",
"CPG4705",
"CPG1925",
"CPG1909",
"CPG1875",
"CPG4473",
"CPG1842",
"CPG1966",
"CPG4697",
"CPG4218",
"CPG1917",
"CPG4721",
"CPG4523",
"CPG4713",
"CPG1958",
"CPG11213",
"CPG4515",
"CPG4481",
"CPG1974",
"CPG8995",
"CPG8888",
"CPG8987",
"CPG8870",
"CPG8862",
"CPG9019",
"CPG9001",
"CPG10462",
"CPG10397",
"CPG10389",
"CPG8979",
"CPG9035",
"CPG9027",
"CPG10348",
"CPG8888",
"CPG8995",
"CPG8987",
"CPG8870",
"CPG9019",
"CPG8862",
"CPG9001",
"CPG10462",
"CPG10389",
"CPG10397",
"CPG8979",
"CPG9035",
"CPG9027",
"CPG10348",
"CPG247312",
"CPG247387",
"CPG247544",
"CPG247841",
"CPG248880",
"CPG247981",
"CPG247601",
"CPG248625",
"CPG248450",
"CPG248476",
"CPG247379",
"CPG249151",
"CPG247965",
"CPG248831",
"CPG248674",
"CPG249052",
"CPG248237",
"CPG249169",
"CPG247650",
"CPG248278",
"CPG247825",
"CPG247494",
"CPG248823",
"CPG248443",
"CPG249060",
"CPG248815",
"CPG248138",
"CPG248583",
"CPG247775",
"CPG247874",
"CPG248542",
"CPG247593",
"CPG248773",
"CPG248294",
"CPG248096",
"CPG248500",
"CPG248393",
"CPG249102",
"CPG248617",
"CPG248971",
"CPG247551",
"CPG248682",
"CPG248898",
"CPG247312",
"CPG248054",
"CPG248146",
"CPG248914",
"CPG247387",
"CPG248690",
"CPG248377",
"CPG247544",
"CPG248286",
"CPG247841",
"CPG248880",
"CPG247981",
"CPG248716",
"CPG248625",
"CPG248450",
"CPG248476",
"CPG247379",
"CPG247965",
"CPG248831",
"CPG249151",
"CPG248237",
"CPG248674",
"CPG249052",
"CPG249169",
"CPG247650",
"CPG248278",
"CPG247825",
"CPG247973",
"CPG247478",
"CPG247460",
"CPG248856",
"CPG247494",
"CPG248963",
"CPG248823",
"CPG248443",
"CPG249060",
"CPG248815",
"CPG248138",
"CPG248583",
"CPG247775",
"CPG248542",
"CPG247874",
"CPG247593",
"CPG248773",
"CPG248294",
"CPG248096",
"CPG249102",
"CPG248393",
"CPG248500",
"CPG247551",
"CPG248971",
"CPG248617",
"CPG248682",
"CPG248518",
"CPG248070",
"CPG247809",
"CPG247320",
"CPG248567",
"CPG248153",
"CPG247817",
"donor6_S0",
"CPG247411",
"CPG248336",
"CPG249011",
"CPG248526",
"CPG247734",
"donor7_S0",
"CPG247932",
"CPG248351",
"CPG247445",
"CPG247486",
"CPG249078",
"CPG247585",
"CPG247742",
"CPG248724",
"CPG249201",
"CPG249144",
"CPG248385",
"CPG248435",
"CPG248757",
"CPG247684",
"CPG248591",
"CPG247536",
"CPG247924",
"CPG247635",
"CPG249177",
"CPG248260",
"CPG248666",
"CPG248658",
"CPG248328",
"CPG248401",
"CPG247346",
"CPG247619",
"CPG247890",
"CPG248765",
"CPG248807",
"CPG247429",
"CPG247767",
"donor7_S0",
"CPG248849",
"CPG248419",
"CPG248039",
"CPG248930",
"CPG247676",
"CPG247700",
"CPG249029",
"CPG248195",
"CPG247916",
"CPG248062",
"CPG248252",
"CPG247833",
"CPG247858",
"CPG248534",
"CPG248708",
"CPG247437",
"CPG247718",
"CPG247692",
"CPG248492",
"CPG249003",
"CPG248245",
"CPG248344",
"CPG248922",
"CPG247528",
"CPG248641",
"CPG247908",
"CPG248708",
"CPG247858",
"CPG248344",
"CPG247437",
"CPG247718",
"CPG249003",
"CPG248534",
"CPG248245",
"CPG247833",
"CPG248492",
"CPG248708",
"CPG247858",
"CPG248344",
"CPG247437",
"CPG247718",
"CPG247833",
"CPG249003",
"CPG248245",
"CPG248534",
"CPG248492",
"CPG248864",
"CPG248484",
"CPG248088",
"CPG248989",
"CPG249136",
"CPG248468",
"CPG248310",
"CPG248575",
"CPG248203",
"CPG248740",
"CPG247999",
"CPG248864",
"CPG248989",
"CPG248088",
"CPG249136",
"CPG248484",
"CPG248468",
"CPG248310",
"CPG248575",
"CPG248203",
"CPG248740",
"CPG247999",
"CPG248021",
"CPG248179",
"CPG249094",
"CPG247866",
"CPG247726",
"CPG247304",
"CPG248211",
"CPG249094",
"CPG248013",
"CPG248021",
"CPG248187",
"CPG248161",
"CPG247726",
"CPG247510",
"CPG247353",
"CPG249128",
"CPG247361",
"CPG248302",
"CPG247452",
"CPG247643",
"CPG248781",
"CPG247866",
"CPG249268",
"CPG248179",
"CPG247296",
"CPG249045",
"CPG247627",
"CPG248229",
"CPG248112",
"CPG247882",
"CPG248732",
"CPG248005",
"CPG247791",
"CPG248047",
"CPG247353",
"CPG248013",
"CPG248187",
"CPG248732",
"CPG248781",
"CPG248302",
"CPG247510",
"CPG247361",
"CPG248229",
"CPG247627",
"CPG248005",
"CPG247353",
"CPG248187",
"CPG248013",
"CPG248732",
"CPG248781",
"CPG248302",
"CPG247510",
"CPG247361",
"CPG248229",
"CPG247627",
"CPG248005",
"CPG255562",
"CPG250696",
"CPG252411",
"CPG253518",
"CPG253682",
"CPG252239",
"CPG254755",
"CPG251330",
"CPG256446",
"CPG252247",
"CPG251496",
"CPG251611",
"CPG251520",
"CPG251074",
"CPG249649",
"CPG305953",
"CPG252510",
"CPG253062",
"CPG305839",
"CPG253252",
"CPG252312",
"CPG251108",
"CPG306167",
"CPG256008",
"CPG252338",
"CPG252635",
"CPG256065",
"CPG254326",
"CPG254904",
"CPG253849",
"CPG249987",
"CPG255497",
"CPG251629",
"CPG252734",
"CPG252957",
"CPG251785",
"CPG250688",
"CPG251033",
"CPG251314",
"CPG252809",
"CPG251470",
"CPG305599",
"CPG250951",
"CPG250795",
"CPG255687",
"CPG255737",
"CPG251702",
"CPG251546",
"CPG249938",
"CPG251058",
"CPG254151",
"CPG256347",
"CPG255356",
"CPG256420",
"CPG249623",
"CPG255810",
"CPG250035",
"CPG251728",
"CPG252353",
"CPG255984",
"CPG253104",
"CPG254805",
"CPG255844",
"CPG252007",
"CPG251660",
"CPG251249",
"CPG250712",
"CPG256032",
"CPG252163",
"CPG251082",
"CPG250530",
"CPG252189",
"CPG253526",
"CPG250811",
"CPG256222",
"CPG254524",
"CPG254474",
"CPG255026",
"CPG252122",
"CPG253807",
"CPG255760",
"CPG252627",
"CPG251777",
"CPG255877",
"CPG251819",
"CPG305821",
"CPG254912",
"CPG250985",
"CPG253435",
"CPG256271",
"CPG305425",
"CPG251637",
"CPG306241",
"CPG252106",
"CPG255133",
"CPG252437",
"CPG250274",
"CPG255257",
"CPG305870",
"CPG251223",
"CPG251207",
"CPG250753",
"CPG253773",
"CPG253823",
"CPG306092",
"CPG250142",
"CPG254938",
"CPG254466",
"CPG305987",
"CPG252783",
"CPG254540",
"CPG254789",
"CPG251405",
"CPG253476",
"CPG252262",
"CPG253971",
"CPG252197",
"CPG251868",
"CPG255661",
"CPG250571",
"CPG253708",
"CPG252726",
"CPG255836",
"CPG252775",
"CPG252031",
"CPG255018",
"CPG254797",
"CPG305466",
"CPG253419",
"CPG250647",
"CPG253096",
"CPG255554",
"CPG255547",
"CPG255158",
"CPG253047",
"CPG306217",
"CPG254771",
"CPG305920",
"CPG250845",
"CPG253070",
"CPG253088",
"CPG255513",
"CPG253633",
"CPG252866",
"CPG253153",
"CPG255323",
"CPG256149",
"CPG255885",
"CPG252569",
"CPG250878",
"CPG253872",
"CPG305417",
"CPG306209",
"CPG306019",
"CPG255711",
"CPG253245",
"CPG305615",
"CPG250936",
"CPG249847",
"CPG252015",
"CPG255109",
"CPG253732",
"CPG250738",
"CPG249896",
"CPG252445",
"CPG306126",
"CPG251363",
"CPG255869",
"CPG252056",
"CPG250399",
"CPG250332",
"CPG254722",
"CPG250563",
"CPG306126",
"CPG251363",
"CPG250399",
"CPG255869",
"CPG250332",
"CPG252056",
"CPG254722",
"CPG255448",
"CPG251579",
"CPG253658",
"CPG253815",
"CPG250381",
"CPG255786",
"CPG250472",
"CPG254870",
"CPG306100",
"CPG251421",
"CPG305763",
"CPG251215",
"CPG250118",
"CPG254748",
"CPG305581",
"CPG256263",
"CPG255471",
"CPG305375",
"CPG252825",
"CPG250308",
"CPG255125",
"CPG251355",
"CPG253922",
"CPG256107",
"CPG306191",
"CPG253443",
"CPG250944",
"CPG254342",
"CPG254508",
"CPG249813",
"CPG255679",
"CPG253021",
"CPG305805",
"CPG253856",
"CPG254821",
"CPG305441",
"CPG250506",
"CPG249854",
"CPG253021",
"CPG255679",
"CPG305805",
"CPG253856",
"CPG254821",
"CPG305441",
"CPG250506",
"CPG256115",
"CPG254995",
"CPG253450",
"CPG255588",
"CPG250282",
"CPG250092",
"CPG254599",
"CPG250597",
"CPG255455",
"CPG253989",
"CPG254573",
"CPG250670",
"CPG306258",
"CPG254839",
"CPG252585",
"CPG251652",
"CPG252387",
"CPG253120",
"CPG249862",
"CPG253781",
"CPG251371",
"CPG252858",
"CPG252346",
"CPG249839",
"CPG250662",
"CPG305540",
"CPG252213",
"CPG252841",
"CPG255299",
"CPG254367",
"CPG249805",
"CPG251025",
"CPG255935",
"CPG255117",
"CPG250498",
"CPG255331",
"CPG253237",
"CPG255364",
"CPG253559",
"CPG249946",
"CPG305508",
"CPG252742",
"CPG254565",
"CPG251587",
"CPG250860",
"CPG305482",
"CPG251991",
"CPG252379",
"CPG255174",
"CPG256123",
"CPG250803",
"CPG249755",
"CPG252791",
"CPG306118",
"CPG255729",
"CPG252916",
"CPG252155",
"CPG250068",
"CPG251751",
"CPG252668",
"CPG251876",
"CPG306068",
"CPG306159",
"CPG251173",
"CPG251843",
"CPG249722",
"CPG252908",
"CPG249748",
"CPG254896",
"CPG255463",
"CPG306233",
"CPG305664",
"CPG251090",
"CPG253674",
"CPG252288",
"CPG250639",
"CPG254425",
"CPG254532",
"CPG253690",
"CPG305433",
"CPG250050",
"CPG252593",
"CPG255604",
"CPG252049",
"CPG249912",
"CPG252718",
"CPG305847",
"CPG250720",
"CPG251272",
"CPG253799",
"CPG253955",
"CPG251439",
"CPG306282",
"CPG252072",
"CPG252759",
"CPG254169",
"CPG253195",
"CPG252882",
"CPG251561",
"CPG305458",
"CPG252932",
"CPG255034",
"CPG251280",
"CPG256255",
"CPG255307",
"CPG249995",
"CPG251694",
"CPG251447",
"CPG305813",
"CPG250779",
"CPG250183",
"CPG305946",
"CPG255315",
"CPG254458",
"CPG250019",
"CPG250886",
"CPG250043",
"CPG305516",
"CPG250423",
"CPG255695",
"CPG252130",
"CPG251454",
"CPG252643",
"CPG250514",
"CPG305979",
"CPG250589",
"CPG255091",
"CPG255521",
"CPG252650",
"CPG253864",
"CPG252999",
"CPG249664",
"CPG249797",
"CPG250225",
"CPG306290",
"CPG252205",
"CPG250522",
"CPG253625",
"CPG251488",
"CPG249706",
"CPG254607",
"CPG305961",
"CPG256073",
"CPG255166",
"CPG306175",
"CPG254581",
"CPG250456",
"CPG252304",
"CPG252452",
"CPG255414",
"CPG254680",
"CPG254441",
"CPG255851",
"CPG253203",
"CPG255653",
"CPG251744",
"CPG252098",
"CPG255950",
"CPG254987",
"CPG251942",
"CPG306134",
"CPG305706",
"CPG250928",
"CPG250605",
"CPG252692",
"CPG305367",
"CPG255083",
"CPG254714",
"CPG305672",
"CPG254029",
"CPG255968",
"CPG255745",
"CPG251595",
"CPG253468",
"CPG250084",
"CPG251389",
"CPG250258",
"CPG252403",
"CPG254664",
"CPG253930",
"CPG252981",
"CPG254631",
"CPG249870",
"CPG249904",
"CPG305532",
"CPG256206",
"CPG250449",
"CPG254615",
"CPG253005",
"CPG249730",
"CPG250266",
"CPG250555",
"CPG256412",
"CPG250290",
"CPG250134",
"CPG250241",
"CPG251041",
"CPG250001",
"CPG255349",
"CPG254359",
"CPG254516",
"CPG251264",
"CPG250829",
"CPG251462",
"CPG253666",
"CPG253740",
"CPG256461",
"CPG256339",
"CPG305607",
"CPG305565",
"CPG254706",
"CPG255901",
"CPG305755",
"CPG255505",
"CPG305904",
"CPG253757",
"CPG251132",
"CPG250977",
"CPG256321",
"CPG255976",
"CPG252536",
"CPG250480",
"CPG251959",
"CPG253591",
"CPG255794",
"CPG251165",
"CPG251199",
"CPG253716",
"CPG250746",
"CPG305631",
"CPG253039",
"CPG252890",
"CPG251231",
"CPG251835",
"CPG249631",
"CPG250167",
"CPG255943",
"CPG251512",
"CPG255927",
"CPG256388",
"CPG253187",
"CPG306225",
"CPG251553",
"CPG254383",
"CPG249888",
"CPG251140",
"CPG250365",
"CPG253765",
"CPG251918",
"CPG250761",
"CPG305573",
"CPG253500",
"CPG252460",
"CPG251009",
"CPG253542",
"CPG256289",
"CPG253179",
"CPG254946",
"CPG252684",
"CPG249672",
"CPG251116",
"CPG305656",
"CPG251934",
"CPG255570",
"CPG253161",
"CPG252965",
"CPG305854",
"CPG252478",
"CPG255778",
"CPG250548",
"CPG256099",
"CPG254763",
"CPG250233",
"CPG251686",
"CPG251256",
"CPG255000",
"CPG305490",
"CPG255380",
"CPG249698",
"CPG249714",
"CPG250613",
"CPG252064",
"CPG254433",
"CPG306266",
"CPG256081",
"CPG254813",
"CPG252551",
"CPG254490",
"CPG252080",
"CPG251983",
"CPG250217",
"CPG251322",
"CPG254557",
"CPG253617",
"CPG250837",
"CPG249920",
"CPG306308",
"CPG251397",
"CPG250704",
"CPG252171",
"CPG254672",
"CPG252817",
"CPG249961",
"CPG255141",
"CPG306076",
"CPG305474",
"CPG249656",
"CPG255539",
"CPG253229",
"CPG249680",
"CPG306043",
"CPG255067",
"CPG250175",
"CPG250324",
"CPG255182",
"CPG252700",
"CPG250407",
"CPG305557",
"CPG250464",
"CPG305912",
"CPG250894",
"CPG256057",
"CPG251975",
"CPG252148",
"CPG250316",
"CPG251504",
"CPG251181",
"CPG252544",
"CPG305524",
"CPG253583",
"CPG254391",
"CPG251967",
"CPG250340",
"CPG251736",
"CPG250415",
"CPG251900",
"CPG252577",
"CPG253724",
"CPG255919",
"CPG252676",
"CPG255893",
"CPG252528",
"CPG250027",
"CPG253534",
"CPG255992",
"CPG252874",
"CPG255273",
"CPG252221",
"CPG253641",
"CPG250076",
"CPG254854",
"CPG254649",
"CPG254730",
"CPG305383",
"CPG254888",
"CPG251017",
"CPG250191",
"CPG305649",
"CPG255638",
"CPG253211",
"CPG253492",
"CPG253906",
"CPG305698",
"CPG306084",
"CPG252320",
"CPG254656",
"CPG306142",
"CPG251827",
"CPG252973",
"CPG305862",
"CPG252601",
"CPG250787",
"CPG256404",
"CPG254409",
"CPG256297",
"CPG251413",
"CPG249789",
"CPG252429",
"CPG251124",
"CPG254847",
"CPG256248",
"CPG255612",
"CPG306274",
"CPG305391",
"CPG251884",
"CPG305680",
"CPG251066",
"CPG255620",
"CPG251645",
"CPG253914",
"CPG253831",
"CPG252361",
"CPG249763",
"CPG255406",
"CPG306035",
"CPG250852",
"CPG254011",
"CPG251801",
"CPG251793",
"CPG305623",
"CPG255752",
"CPG256396",
"CPG253146",
"CPG252502",
"CPG252395",
"CPG253427",
"CPG251850",
"CPG250621",
"CPG254060",
"CPG305938",
"CPG256230",
"CPG253997",
"CPG254417",
"CPG252023",
"CPG252940",
"CPG250993",
"CPG252619",
"CPG252114",
"CPG251710",
"CPG253963",
"CPG250126",
"CPG305722",
"CPG250969",
"CPG250654",
"CPG250209",
"CPG253013",
"CPG255802",
"CPG251678",
"CPG254375",
"CPG253112",
"CPG249771",
"CPG306050",
"CPG255596",
"CPG253054",
"CPG306183",
"CPG251926",
"CPG254920",
"CPG256453",
"CPG254862",
"CPG249953",
"CPG253575",
"CPG251538",
"CPG305714",
"CPG252296",
"CPG250373",
"CPG256438",
"CPG249979",
"CPG251157",
"CPG254482",
"CPG306027",
"CPG254623",
"CPG255489",
"CPG256131",
"CPG255430",
"CPG254334",
"CPG253948",
"CPG253567",
"CPG252924"}

In [46]:
missing_ids = cpg_ids_prev.difference(set(adata_240.obs['cpg_id']).union(set(adata_240.obs['cpg_id_old'])))

In [47]:
len(missing_ids)

86

In [48]:
missing_ids

{'CPG10173',
 'CPG10207',
 'CPG10215',
 'CPG10223',
 'CPG10231',
 'CPG10256',
 'CPG10264',
 'CPG10272',
 'CPG10298',
 'CPG10306',
 'CPG10314',
 'CPG10322',
 'CPG10363',
 'CPG10371',
 'CPG10389',
 'CPG10413',
 'CPG10421',
 'CPG10439',
 'CPG10447',
 'CPG10454',
 'CPG10504',
 'CPG10579',
 'CPG10595',
 'CPG10611',
 'CPG10637',
 'CPG10678',
 'CPG10694',
 'CPG10710',
 'CPG10751',
 'CPG10777',
 'CPG10793',
 'CPG10801',
 'CPG10827',
 'CPG10835',
 'CPG10843',
 'CPG10850',
 'CPG10868',
 'CPG10876',
 'CPG10892',
 'CPG10918',
 'CPG10942',
 'CPG11031',
 'CPG11056',
 'CPG11064',
 'CPG11130',
 'CPG11155',
 'CPG11221',
 'CPG11254',
 'CPG11270',
 'CPG11288',
 'CPG1792',
 'CPG2212',
 'CPG3111',
 'CPG4531',
 'CPG4994',
 'CPG5066',
 'CPG6379',
 'CPG6817',
 'CPG6932',
 'CPG7013',
 'CPG7096',
 'CPG7161',
 'CPG7229',
 'CPG7682',
 'CPG7716',
 'CPG7740',
 'CPG7989',
 'CPG8045',
 'CPG8102',
 'CPG8177',
 'CPG8219',
 'CPG8466',
 'CPG8664',
 'CPG8714',
 'CPG8748',
 'CPG8763',
 'CPG8847',
 'CPG8904',
 'CPG8987',
 '

In [54]:
adata_240.obs[adata_240.obs['sequencing_library'] == 'S0011a'][['sequencing_library', 'cpg_id', 'cpg_id_old']].value_counts()

sequencing_library  cpg_id     cpg_id_old
S0011a              CPG311852  CPG3855       1265
                    CPG317800  CPG11007      1075
                    CPG313924  CPG6056        988
                    CPG311803  CPG3806        932
                    CPG311845  CPG3848        925
                    CPG316562  CPG9225        916
                    CPG317826  CPG11023       876
                    CPG311860  CPG3863        836
                    CPG317818  CPG11015       834
                    CPG311993  CPG4010        815
                    CPG317784  CPG10983       807
                    CPG313932  CPG6064        748
                    CPG317834  CPG11049       734
                    CPG313288  CPG5389        706
                    CPG313981  CPG6122        700
                    CPG311837  CPG3830        677
                    CPG312421  CPG4465        658
                    CPG313973  CPG6114        584
                    CPG317792  CPG10991       563
        

### 224 samples object

In [4]:
adata_224 = sc.read('/directflow/SCCGGroupShare/projects/anncuo/TenK10K_pilot/tenk10k/data_processing/integrated_objects/224_libraries/concatenated_gene_info_donor_info.h5ad')

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


In [5]:
adata_224.obs.shape

(4494637, 40)

In [6]:
# how many pools?
adata_224.obs['batch'].nunique()

224

In [9]:
[print(i) for i in adata_224.obs['individual'].unique()]

CPG314682
CPG314286
CPG314690
CPG309070
CPG314310
CPG310961
CPG309088
CPG309096
CPG310920
CPG314633
CPG311951
CPG310979
CPG310938
CPG314328
CPG314641
CPG314625
CPG310995
CPG310953
CPG314658
CPG311969
CPG310946
CPG314302
CPG310912
CPG314294
CPG310904
CPG310987
CPG316620
CPG314518
CPG314526
CPG317016
CPG316646
CPG314492
CPG314542
CPG314443
CPG316612
CPG316679
CPG314476
CPG314500
CPG316638
CPG314450
CPG316653
CPG316695
CPG316604
CPG314534
CPG314468
CPG316661
CPG314567
CPG314559
CPG316596
CPG314435
CPG316687
CPG316711
CPG316703
CPG314484
CPG314229
CPG309245
CPG310565
CPG314195
CPG309005
CPG314245
CPG314237
CPG314757
CPG310607
CPG309286
CPG314609
CPG310599
CPG314260
CPG314278
CPG314187
CPG309252
CPG309260
CPG310573
CPG309013
CPG314617
CPG309278
CPG314252
CPG314203
CPG309237
CPG314211
CPG310581
CPG314740
CPG314732
CPG314807
CPG314575
CPG317966
CPG314120
CPG314419
CPG317958
CPG314427
CPG314179
CPG314112
CPG314781
CPG313668
CPG314393
CPG314385
CPG314815
CPG313882
CPG314799
CPG314401
CPG314823


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [11]:
adata_224.obs['individual'].nunique()

1654

In [14]:
set(ind_224).difference(set(ind_240))

{'donor6_S0049', 'donor7_S0049', 'donor7_S0053'}

In [ ]:
# how many individuals?
adata_224.obs['sequencing_library'].nunique()

In [ ]:
# compare donor list to see which ones are missing 

In [ ]:
adata_224.obs['individual'].nunique()

In [16]:
5084027 - 4494637

589390

In [ ]:
adata_224.obs.columns

In [ ]:
adata_224.obs.columns.nunique()

In [9]:
obs_240 = ['cellbender_background_fraction', 'cellbender_cell_probability',
       'cellbender_cell_size', 'cellbender_droplet_efficiency',
       'celltypist_predicted_labels', 'celltypist_over_clustering',
       'celltypist_majority_voting', 'celltypist_conf_score', 'wg2_sample',
       'wg2_nCount_RNA', 'wg2_nFeature_RNA', 'wg2_percent_mt',
       'wg2_azimuth_predicted_celltype_l2',
       'wg2_azimuth_predicted_celltype_l2_score', 'wg2_scpred_prediction',
       'Vireo_Individual_Assignment', 'Vireo_DropletType',
       'scDblFinder_DropletType', 'scDblFinder_Score', 'scds_score',
       'scds_DropletType', 'MajoritySinglet_DropletType',
       'MajoritySinglet_Individual_Assignment', 'n_genes', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'original_barcode',
       'new_cell_name', 'sequencing_library', 'individual', 'cohort', 'ct_id',
       'cpg_id', 'onek1k_id', 'cpg_id_old', 'tob_id', 'onek1k_donor', 'batch']
obs_224 = ['cellbender_background_fraction', 'cellbender_cell_probability',
       'cellbender_cell_size', 'cellbender_droplet_efficiency',
       'celltypist_predicted_labels', 'celltypist_over_clustering',
       'celltypist_majority_voting', 'celltypist_conf_score', 'wg2_sample',
       'wg2_nCount_RNA', 'wg2_nFeature_RNA', 'wg2_percent_mt',
       'wg2_azimuth_predicted_celltype_l2',
       'wg2_azimuth_predicted_celltype_l2_score', 'wg2_scpred_prediction',
       'Vireo_Individual_Assignment', 'Vireo_DropletType',
       'scDblFinder_DropletType', 'scDblFinder_Score', 'scds_score',
       'scds_DropletType', 'MajoritySinglet_DropletType',
       'MajoritySinglet_Individual_Assignment', 'n_genes', 'n_genes_by_counts',
       'total_counts', 'total_counts_mt', 'pct_counts_mt', 'original_barcode',
       'new_cell_name', 'sequencing_library', 'individual', 'cohort',
       'onek1k_id', 'cpg_id_old', 'tob_id', 'cpg_id', 'onek1k_donor', 'ct_id',
       'batch']

In [10]:
# checking if any columns are missing compared to the old version 
[print(i) for i in obs_224 if i not in obs_240]

[]

In [11]:
4827648 - 4494637

333011

In [15]:
[print(i) for i in adata_240.obs['individual'].unique()]

CPG248914
CPG248146
CPG247312
CPG248690
CPG247387
CPG247841
CPG248880
CPG248286
CPG248054
CPG248377
CPG247981
CPG247544
CPG248716
CPG313536
CPG313551
CPG311720
CPG313544
CPG317768
CPG317735
CPG311746
CPG313890
CPG313957
CPG311779
CPG317917
CPG311795
CPG311753
CPG313569
CPG311761
CPG313874
CPG309930
CPG313908
CPG311787
CPG313916
CPG317750
CPG311738
CPG317776
CPG317743
CPG313510
CPG313528
CPG309112
CPG313866
CPG309104
CPG310896
CPG313841
CPG313817
CPG313767
CPG310862
CPG310847
CPG313486
CPG313759
CPG313627
CPG310888
CPG313858
CPG310797
CPG313494
CPG313643
CPG313825
CPG313742
CPG310870
CPG313833
CPG310854
CPG255448
CPG254870
CPG251579
CPG255786
CPG250381
CPG253815
CPG253658
CPG250472
CPG314138
CPG314765
CPG314732
CPG314427
CPG314393
CPG314401
CPG317966
CPG314807
CPG314179
CPG314385
CPG314575
CPG314781
CPG314112
CPG317958
CPG314120
CPG314815
CPG314823
CPG314419
CPG314740
CPG313668
CPG313882
CPG314799
CPG314773
CPG315010
CPG315028
CPG312025
CPG309427
CPG314997
CPG314872
CPG311977
CPG314831


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [17]:
[print(i) for i in adata_240.obs['sequencing_library'].unique()]

S0044
S0034b
S0017b
S0084
S0024a
S0019c
S0025_28b
S0020a
S0022c
S0033b
S0034a
S0018a
S0032c
S0031c
S0033c
S0024b
S0030a
S0067
S0031b
S0124
S0036b
S0021c
S0020c
S0032a
S0142
S0021_24a
S0030_33b
S0033a
S0022a
S0086
S0001c
S0094_re
S0034_37a
S0099
S0083_re
S0117
S0090
S0088
S0095
S0100
S0091
S0101
S0087
S0089
S0149
S0057a
S0104
S0056a
S0059a
S0102
S0152
S0056b
S0150
S0057b
S0153
S0059b
S0146
S0058_re
S0148
S0007a
S0068
S0041
S0072
S0039b
S0137
S0009c
S0135
S0038a
S0119
S0065
S0010a
S0037c
S0039a
S0107
S0043
S0105
S0045
S0048
S0034_37b
S0098
S0047
S0038c
S0085
S0132
S0055
S0092
S0063
S0076
S0097
S0009a
S0106
S0108
S0003a
S0116
S0145
S0004a
S0014b
S0003c
S0010c
S0005a
S0054
S0128
S0138
S0121
S0130
S0129
S0002c
S0013a
S0127
S0115
S0004b
S0143
S0001b
S0052
S0014c
S0014a
S0015c
S0010b
S0125
S0013c
S0011a
S0112
S0006a
S0088_re
S0046
S0042_repeat
S0110
S0012c
S0096
S0050
S0071
S0061
S0114
S0120
S0064
S0131
S0056
S0003b
S0144
S0134
S0001a
S0070
S0008c
S0016a
S0015b
S0073
S0049
S0083
S0075
S0019b


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [26]:
adata_240.obs[['sequencing_library', 'individual']].groupby(['sequencing_library', 'individual']).size().reset_index(name='counts')

,sequencing_library,individual,counts
0,S0001a,CPG247296,0
1,S0001a,CPG247304,0
2,S0001a,CPG247312,0
3,S0001a,CPG247320,0
4,S0001a,CPG247346,0
...,...,...,...
414715,S0153,CPG317982,0
414716,S0153,CPG317990,0
414717,S0153,donor6_S0049,0
414718,S0153,donor7_S0049,0


In [27]:
adata_240.obs

,cellbender_background_fraction,cellbender_cell_probability,cellbender_cell_size,cellbender_droplet_efficiency,celltypist_predicted_labels,celltypist_over_clustering,celltypist_majority_voting,celltypist_conf_score,wg2_sample,wg2_nCount_RNA,...,sequencing_library,individual,cohort,ct_id,cpg_id,onek1k_id,cpg_id_old,tob_id,onek1k_donor,batch
AAACCCAAGACAGCGT_S0044-0,0.002164,0.999955,9000.090820,1.141465,CD16- NK cells,137,Tem/Trm cytotoxic T cells,0.647146,S0044,9703.0,...,S0044,CPG248914,BioHEART,CT_959,CPG248914,NaN,NaN,NaN,NaN,0
AAACCCAAGATGACCG_S0044-0,0.001005,0.999955,9096.833008,1.273903,Tcm/Naive helper T cells,177,Tcm/Naive helper T cells,0.996452,S0044,10941.0,...,S0044,CPG248146,BioHEART,CT_469,CPG248146,NaN,NaN,NaN,NaN,0
AAACCCAAGGGACACT_S0044-0,0.001777,0.999955,9304.674805,1.150571,Tem/Effector helper T cells,27,Tem/Trm cytotoxic T cells,0.979878,S0044,10130.0,...,S0044,CPG247312,BioHEART,CT_113,CPG247312,NaN,NaN,NaN,NaN,0
AAACCCAAGGGCAGTT_S0044-0,0.024942,0.999972,6600.250000,0.783240,Tem/Temra cytotoxic T cells,0,Tem/Temra cytotoxic T cells,0.999366,S0044,4771.0,...,S0044,CPG247312,BioHEART,CT_113,CPG247312,NaN,NaN,NaN,NaN,0
AAACCCACAACCGGAA_S0044-0,0.004508,0.999778,8140.067871,0.899632,Tem/Temra cytotoxic T cells,119,Tem/Temra cytotoxic T cells,0.998791,S0044,6877.0,...,S0044,CPG248146,BioHEART,CT_469,CPG248146,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGGTTCAAAGA_S0025b-239,0.003661,0.999883,5989.370117,1.090966,Tcm/Naive helper T cells,133,Tcm/Naive helper T cells,0.999987,S0025b,6009.0,...,S0025b,CPG316638,TOB,NaN,CPG316638,877_878,CPG9290,TOB1010,878,239
TTTGTTGGTTCTAAGC_S0025b-239,0.008182,0.999894,5166.430176,0.861364,Tem/Temra cytotoxic T cells,247,Tem/Temra cytotoxic T cells,0.856409,S0025b,4033.0,...,S0025b,CPG314542,TOB,NaN,CPG314542,618_619,CPG6718,TOB1474,619,239
TTTGTTGTCCCAATAG_S0025b-239,0.002411,0.999955,6014.755859,1.806147,CD16+ NK cells,33,CD16+ NK cells,0.999956,S0025b,9953.0,...,S0025b,CPG314500,TOB,NaN,CPG314500,614_615,CPG6676,TOB1470,615,239
TTTGTTGTCGAACGGA_S0025b-239,0.006890,0.999894,6003.712891,0.893320,Tcm/Naive helper T cells,246,Tcm/Naive helper T cells,0.999953,S0025b,4935.0,...,S0025b,CPG314500,TOB,NaN,CPG314500,614_615,CPG6676,TOB1470,615,239
